In [1]:
#core enviroment libraries for RL 
import gym
from gym.spaces import Discrete, MultiDiscrete,Box, Dict, MultiBinary,Tuple
#mo_gym is for creating enviroments with multiple reward functions simultaneously 
import mo_gymnasium as mo_gym#we are going to phase out gym in favor of mo_gym

#utilities 
import numpy as np
import random
#these libraries have to do with the agents 
import ray
from ray.rllib.utils.deprecation import Deprecated
from ray.rllib.utils.metrics import (
    LAST_TARGET_UPDATE_TS,
    NUM_AGENT_STEPS_SAMPLED,
    NUM_ENV_STEPS_SAMPLED,
    NUM_TARGET_UPDATES,
    SYNCH_WORKER_WEIGHTS_TIMER,
)
from ray.rllib.utils.replay_buffers.utils import sample_min_n_steps_from_buffer
from ray.rllib.utils.typing import ResultDict, AlgorithmConfigDict
from ray.rllib.utils.deprecation import DEPRECATED_VALUE
from ray.rllib.utils.deprecation import deprecation_warning
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray import air, tune

from nebulgym.decorators.torch_decorators import accelerate_model, accelerate_dataset
import argparse
import logging
import os
#import pathpy as pp
from ray.tune import register_env
from ray.rllib.algorithms.qmix import QMixConfig
#from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.examples.env.multi_agent import MultiAgentCartPole
import mne# preprocessing and brain importation and utilities library including acessing and preprocessing the EEG data
#these libraries have to do with the free energy principle
#import pymdp
#from pymdp import utils
#from pymdp.agent import Agent
#from gym.spaces import 

#optimization of deep learning and RL aspects of algorithm these will allow the algorithm to run faster with less memory 
#from composer import Trainer
#from nebullvm.api.functions import optimize_model 
from numba import jit
import functools

import taichi as ti
import taichi.math as tm
"""
dependency network

Qmix.py - has qmixpolicy.py as a dependency 
Qmixpolicy.py has  mixers.py and Model.py dependencies
Model.py -base
mixers.py -base

if we do this carefully we can use taichi to carry out speedup

The mixer and the bmodel would be ti.funcs

qmixpolicy would also be a ti.func

qmix would be the ti.kernel

"""

import tensorflow as tf

import ivy# library for interoperable across all deep learning frameworks 
#import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import torch
from laplace import Laplace #for model selection 
from laplace.baselaplace import FullLaplace
from laplace.curvature.backpack import BackPackGGN

#below libraries are core libraries for q-mix Rllib algorithm
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.preprocessors import get_preprocessor
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override
from torch import nn

from ray.rllib.policy.torch_policy import TorchPolicy

from typing import Optional, Type,  Dict, List, Tuple

from ray.rllib.algorithms.simple_q.simple_q import SimpleQ, SimpleQConfig
from ray.rllib.algorithms.qmix.qmix_policy import QMixTorchPolicy
from ray.rllib.utils.replay_buffers.utils import update_priorities_in_replay_buffer
from ray.rllib.execution.rollout_ops import (
    synchronous_parallel_sample,
)

from ray.rllib.execution.train_ops import (
    multi_gpu_train_one_step,
    train_one_step,
)
from ray.rllib.policy.policy import Policy
import tree  # pip install dm_tree

from ray.rllib.algorithms.qmix.mixers import VDNMixer, QMixer
#from ray.rllib.algorithms.qmix.model import RNNModel, _get_size
from ray.rllib.env.multi_agent_env import MultiAgentEnv, ENV_STATE, make_multi_agent
from ray.rllib.env.wrappers.group_agents_wrapper import GROUP_REWARDS
from ray.rllib.models.catalog import ModelCatalog
from ray.rllib.models.modelv2 import _unpack_obs
from ray.rllib.models.torch.torch_action_dist import TorchCategorical
from ray.rllib.policy.rnn_sequencing import chop_into_sequences
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.metrics.learner_info import LEARNER_STATS_KEY
from ray.rllib.utils.typing import TensorType

from ray.tune.registry import get_trainable_cls

import pyro
import pyro.distributions as dist
from pyro.contrib.oed.eig import posterior_eig
from pyro.optim import Adam
from pyro.infer.abstract_infer import TracePosterior
import nitime
from deeptime.sindy import SINDy

#non causal counterfactuals
import dice_ml
from dice_ml.utils import helpers # helper functions

#data visualization
import pygwalker as pyg

"""
from github3 import login, GitHub
from getpass import getpass, getuser
import sys

"""

ti.init(arch=ti.gpu)

C:\Users\subar\anaconda3\envs\d\lib\site-packages\flax\struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
C:\Users\subar\anaconda3\envs\d\lib\site-packages\flax\struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
C:\Users\subar\anaconda3\envs\d\lib\site-packages\nebulgym\training_learners\model_engines\ort_engine.py:14: UserWarning: No ONNXRuntime training library for pytorch has been detected. The ORT backend won't be used.
  warnings.warn(
C:\Users\subar\anaconda3\envs\d\lib\site-packages\nebulgym\training_learners\model_engines\rammer_engine.py:16: UserWarning: No valid Rammer installation found. Using the Rammer backend will result in an error.
  

[Taichi] version 1.6.0, llvm 15.0.1, commit f1c6fbbd, win, python 3.9.16
[Taichi] Starting on arch=cuda


In [30]:
from ursina import *            # this will import everything we need from ursina with just one line.

app = Ursina()
ground = Entity(
    model = 'cube',
    color = color.magenta,
    z = -.1,
    y = -3,
    origin = (0, .5),
    scale = (50, 1, 10),
    collider = 'box',
    )

app.run()                       # opens a window and starts the game.

package_folder: C:\Users\subar\anaconda3\envs\d\lib\site-packages\ursina
asset_folder: .
1.0
screen resolution: (2560, 1440)
os: Windows
development mode: True
application successfully started
info: changed aspect ratio: 1.778 -> 1.778


SystemExit: 

C:\Users\subar\anaconda3\envs\d\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
import os
os.environ["KERAS_BACKEND"] = "jax"

import keras_core as keras

Using JAX backend.


In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from hummingbird.ml import convert, load

# Create some random data for binary classification
num_classes = 2
X = np.random.rand(100000, 28)
y = np.random.randint(num_classes, size=100000)

# Create and train a model (scikit-learn RandomForestClassifier in this case)
skl_model = RandomForestClassifier(n_estimators=10, max_depth=10)
skl_model.fit(X, y)

# Use Hummingbird to convert the model to PyTorch
model = convert(skl_model, 'pytorch')

# Run predictions on CPU
model.predict(X)

# Run predictions on GPU
model.to('cuda')
model.predict(X)


array([1, 1, 1, ..., 1, 0, 1], dtype=int64)

In [2]:
from jax.experimental.ode import odeint
#
#from jax.random import PRNGKey

import numpyro
import numpyro.distributions as dist
from numpyro.examples.datasets import LYNXHARE, load_dataset
from numpyro.infer import MCMC, NUTS, Predictive
from numpyro.infer import Predictive, SVI, Trace_ELBO

In [1]:
import jax
import jax.numpy as jnp

x = jnp.arange(10)
print(x)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


[0 1 2 3 4 5 6 7 8 9]


In [2]:
#create this like the outcome predictor. This is the expected free energy sub-module ESM
#this will essentially calculate the q(x|y)
#@profile
class ESM(nn.Module,ivy.Module):
     def __init__(self):     

        super().__init__()
        tfk = tf.keras
        tfkl = tf.keras.layers
        input_shape=2
        tfpl = tfp.layers
        prior = tfp.distributions.Dirichlet(concentration)
        prior_entropy = dist.Bernoulli(prior_w_prob).entropy()
        tfd = tfp.distributions
        encoded_size = 16
        #model = tf.keras.Sequential([
        u = tfkl.InputLayer(input_shape=input_shape),
        """
        u = tf.keras.layers.LSTM(25,kernel_initializer='zeros',activation='tanh', dtype = x.dtype, use_bias=True)(u),
        u = tfp.layers.VariationalGaussianProcess(
                num_inducing_points=num_inducing_points, kernel_provider=RBFKernelFn(dtype=x.dtype) , event_shape=(1,),
                inducing_index_points_initializer=tf.compat.v1.constant_initializer(
                    np.linspace(0,x_range, num=1125,
                                dtype=x.dtype)[..., np.newaxis]), unconstrained_observation_noise_variance_initializer=(tf.compat.v1.constant_initializer(np.log(np.expm1(1.)).astype(x.dtype))),variational_inducing_observations_scale_initializer=(tf.compat.v1.constant_initializer(np.log(np.expm1(1.)).astype(x.dtype))), mean_fn=None,
                jitter=1e-06, convert_to_tensor_fn=tfp.distributions.Distribution.sample)(u)

        """

        self.h1 = nn.Linear(51, 64)
        self.h2 = nn.Linear(64, 64)
        self.h3 = nn.Linear(64, 1)

        #in unconstrained thing replace astype with tf.dtype thing.    #tf.initializers.constant(-10.0)
        #])
        def compute_probability(self, y):
            #fk.Model()
            z = nn.functional.relu(self.h1(y))
            z = nn.functional.relu(self.h2(z))
            #o = tf.nn.relu(u)
            return self.h3(z)
        def forward(self, y_dict, design, observation_labels, target_labels):
            pyro.module("posterior_guide", self)

            y = y_dict["y"]

            #bmodel = FullLaplace(bmodel,'regression',prior_precision=2)
            dem_prob = self.compute_probability(y).squeeze()
            pyro.sample("w", dist.Bernoulli(logits=dem_prob))

In [4]:
"""
from pyro.contrib.oed.eig import marginal_eig
def ES(design, observation_labels, target_labels):
    # This shape allows us to learn a different parameter for each candidate design l
    q_logit = pyro.param("q_logit", torch.zeros(design.shape[-2:])).entropy()
    pyro.sample("y", dist.Bernoulli(logits=q_logit).to_event(1))
"""

'\nfrom pyro.contrib.oed.eig import marginal_eig\ndef ES(design, observation_labels, target_labels):\n    # This shape allows us to learn a different parameter for each candidate design l\n    q_logit = pyro.param("q_logit", torch.zeros(design.shape[-2:])).entropy()\n    pyro.sample("y", dist.Bernoulli(logits=q_logit).to_event(1))\n'

In [3]:
#@profile
def memoize(fn=None, **kwargs):
    if fn is None:
        return lambda _fn: memoize(_fn, **kwargs)
    return functools.lru_cache(**kwargs)(fn)

In [4]:
#@profile
class HashingMarginal(dist.Distribution):
    """
    :param trace_dist: a TracePosterior instance representing a Monte Carlo posterior

    Marginal histogram distribution.
    Turns a TracePosterior object into a Distribution
    over the return values of the TracePosterior's model.
    """

    def __init__(self, trace_dist, sites=None):
        assert isinstance(
            trace_dist, TracePosterior
        ), "trace_dist must be trace posterior distribution object"

        if sites is None:
            sites = "_RETURN"

        assert isinstance(sites, (str, list)), "sites must be either '_RETURN' or list"

        self.sites = sites
        super().__init__()
        self.trace_dist = trace_dist

    has_enumerate_support = True

    @memoize(maxsize=10)
    def _dist_and_values(self):
        # XXX currently this whole object is very inefficient
        values_map, logits = collections.OrderedDict(), collections.OrderedDict()
        for tr, logit in zip(self.trace_dist.exec_traces, self.trace_dist.log_weights):
            if isinstance(self.sites, str):
                value = tr.nodes[self.sites]["value"]
            else:
                value = {site: tr.nodes[site]["value"] for site in self.sites}
            if not torch.is_tensor(logit):
                logit = torch.tensor(logit)

            if torch.is_tensor(value):
                value_hash = hash(value.cpu().contiguous().numpy().tobytes())
            elif isinstance(value, dict):
                value_hash = hash(self._dict_to_tuple(value))
            else:
                value_hash = hash(value)
            if value_hash in logits:
                # Value has already been seen.
                logits[value_hash] = dist.util.logsumexp(
                    torch.stack([logits[value_hash], logit]), dim=-1
                )
            else:
                logits[value_hash] = logit
                values_map[value_hash] = value

        logits = torch.stack(list(logits.values())).contiguous().view(-1)
        logits = logits - dist.util.logsumexp(logits, dim=-1)
        d = dist.Categorical(logits=logits)
        return d, values_map

    def sample(self):
        d, values_map = self._dist_and_values()
        ix = d.sample()
        return list(values_map.values())[ix]

    def log_prob(self, val):
        d, values_map = self._dist_and_values()
        if torch.is_tensor(val):
            value_hash = hash(val.cpu().contiguous().numpy().tobytes())
        elif isinstance(val, dict):
            value_hash = hash(self._dict_to_tuple(val))
        else:
            value_hash = hash(val)
        return d.log_prob(torch.tensor([list(values_map.keys()).index(value_hash)]))

    def enumerate_support(self):
        d, values_map = self._dist_and_values()
        return list(values_map.values())[:]

    def _dict_to_tuple(self, d):
        """
        Recursively converts a dictionary to a list of key-value tuples
        Only intended for use as a helper function inside HashingMarginal!!
        May break when keys cant be sorted, but that is not an expected use-case
        """
        if isinstance(d, dict):
            return tuple([(k, self._dict_to_tuple(d[k])) for k in sorted(d.keys())])
        else:
            return d

    def _weighted_mean(self, value, dim=0):
        weights = self._log_weights.reshape([-1] + (value.dim() - 1) * [1])
        max_weight = weights.max(dim=dim)[0]
        relative_probs = (weights - max_weight).exp()
        return (value * relative_probs).sum(dim=dim) / relative_probs.sum(dim=dim)

    @property
    def mean(self):
        samples = torch.stack(list(self._dist_and_values()[1].values()))
        return self._weighted_mean(samples)

    @property
    def variance(self):
        samples = torch.stack(list(self._dist_and_values()[1].values()))
        deviation_squared = torch.pow(samples - self.mean, 2)
        return self._weighted_mean(deviation_squared)

In [5]:
#@profile
def Marginal(fn=None, **kwargs):
    if fn is None:
        return lambda _fn: Marginal(_fn, **kwargs)
    return memoize(
        lambda *args: HashingMarginal(BestFirstSearch(fn, **kwargs).run(*args))
    )

In [7]:
#@ti.data_oriented
class Search(TracePosterior):
    """
    Exact inference by enumerating over all possible executions
    """

    def __init__(self, model, max_tries=int(1e6), **kwargs):
        self.model = model
        self.max_tries = max_tries
        super().__init__(**kwargs)
    #@ti.kernel
    def _traces(self, *args, **kwargs):
        q = queue.Queue()
        q.put(poutine.Trace())
        p = poutine.trace(poutine.queue(self.model, queue=q, max_tries=self.max_tries))
        while not q.empty():
            tr = p.get_trace(*args, **kwargs)
            yield tr, tr.log_prob_sum()
#@ti.data_oriented
class BestFirstSearch(TracePosterior):
    """
    Inference by enumerating executions ordered by their probabilities.
    Exact (and results equivalent to Search) if all executions are enumerated.
    """

    def __init__(self, model, num_samples=None, **kwargs):
        if num_samples is None:
            num_samples = 100
        self.num_samples = num_samples
        self.model = model
        super().__init__(**kwargs)
    #@ti.kernel
    def _traces(self, *args, **kwargs):
        q = queue.PriorityQueue()
        # add a little bit of noise to the priority to break ties...
        q.put((torch.zeros(1).item() - torch.rand(1).item() * 1e-2, poutine.Trace()))
        q_fn = pqueue(self.model, queue=q)
        for i in range(self.num_samples):
            if q.empty():
                # num_samples was too large!
                break
            tr = poutine.trace(q_fn).get_trace(*args, **kwargs)  # XXX should block
            yield tr, tr.log_prob_sum()

In [8]:
#pragmatic value function 

@Marginal
def speaker(state):
    alpha = 1.
    with poutine.scale(scale=torch.tensor(alpha)):
        utterance = utterance_prior()
        pyro.sample("listener", literal_listener(utterance), obs=state)
    return utterance

In [9]:
from dadaptation import DAdaptAdaGrad

In [10]:
optimizer =  DAdaptAdaGrad#torch.optim.SGD
scheduler = pyro.optim.ExponentialLR({'optimizer': optimizer, 'optim_args': {}, 'gamma': 0.1})

In [11]:
@accelerate_model()
class emodel(TorchModelV2, nn.Module,ivy.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):#(self, obs_space, action_space, num_outputs, model_config):
        TorchModelV2.__init__(
             self, obs_space, action_space, num_outputs, model_config, name#self, obs_space, action_space, num_outputs, model_config
        )
        nn.Module.__init__(self)
        self.obs_size = _get_size(obs_space)
        self.rnn_hidden_dim = model_config["lstm_cell_size"]

        """
        pseudo code
        
        assign a prior to be the posterior of the output of previous agent
        
        if no output of agent is found we have a preset prior. Prior is going to be a dirchlet
        
        pragmatic value is probability of outcomes averaged over all policies or E(lnP*o|c) where c is prior
        policies with outcomes close to the prior become more probable
        
        """
        
        #our job here is to minimize the log probability of y given prior. 
        #input_dict["obs_flat"].float()
        #encoder = tfk.Sequential([
        """
        i = tfkl.InputLayer(input_shape=input_shape)
        x = tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
               activation=None)(i)
        z = tfp.layers.VariationalGaussianProcess(
                num_inducing_points=num_inducing_points, kernel_provider=RBFKernelFn(dtype=x.dtype) , event_shape=(1,),
                inducing_index_points_initializer=tf.compat.v1.constant_initializer(
                    np.linspace(0,x_range, num=1125,
                                dtype=x.dtype)[..., np.newaxis]), unconstrained_observation_noise_variance_initializer=(tf.compat.v1.constant_initializer(np.log(np.expm1(1.)).astype(x.dtype))),variational_inducing_observations_scale_initializer=(tf.compat.v1.constant_initializer(np.log(np.expm1(1.)).astype(x.dtype))), mean_fn=None,
                jitter=1e-06, convert_to_tensor_fn=tfp.distributions.Distribution.sample)(x)

        #in unconstrained thing replace astype with tf.dtype thing.    #tf.initializers.constant(-10.0)

        #negloglik = lambda x, rv_x: -rv_x.log_prob(x)


        eig = marginal_eig(model,
                           candidate_designs,       # design, or in this case, tensor of possible designs
                           "y",                     # site label of observations, could be a list
                           "theta",                 # site label of 'targets' (latent variables), could also be list
                           num_samples=100,         # number of samples to draw per step in the expectation
                           num_steps=num_steps,     # number of gradient steps
                           guide=marginal_guide,    # guide q(y)
                           optim=optimizer,         # optimizer with learning rate decay
                           final_num_samples=10000  # at the last step, we draw more samples
                                                    # for a more accurate EIG estimate
                          )

        expected free energy is minimized when when observations are slected that cause a large change in beliefs 
        
        Expected free energy will consist of 3 components
        
        ESM -> q(y|d)
        |
        emodels -> initialization pragmatic value neural network. 
        This is going to be neural network for 
        log(p(y|prior)) +epistemic value  emodels + (epistemic value = ESM - expected ambiguity)
        |
        EFE -> expected free energy/temperature   
        
        pragmatic value will be calculated using a variational gaussian process regression tensorflow neural network 
        
        we are going to explore The Rational Speech Act framework and its relation to calculating the pragmatic value
        
        We will actually calculate both epistemic value and define the neural network for pragmatic value in emodels
        
        but inputs, parameterization and outputs of pragmatic value network will happen on EFE
        
        Next objective: determine the probaility of hidden state given observation
        
        we should use a neural network to calculate the probability of reward given action 
        
        if there is 0 correlation, that is to say reward does not change given a certain action then p(s|y) = 0
        
        pragmatic value observations expecteation of log probability of outcomes given prior 
        
        theta = s=w=x is the hidden latent variable state. Here it is going to be the reward
        
        design is going to be action, outcome is observation, state is the reward
        
        also -H(P(s)) = E(lnP(s))
        
        in EFE we are going to use a relaxed bernoulli with temperature for distribution of entropy of p(o|s)
 
        average predictive entropy = H(posteririor pdf) = entropy of entire epsitemic value 
        
        for our purposes p(theta) = q(theta|design)
        
        pragmatic value is -E(ln(p(y|prior)) which could be converted to prior entropy 
        
        we will implement enhanced expected free energy later to include learning. 
        
        we for now can set the pragmatic value to 0. this is used to set objectives 
        
        NOte: to clear up confusion theta can refer to states or to beliefs about parameters in model 
        
        later for enhanced expected free energy we will have to include theta side by side with states outcomes,
        designs and prior 
        
        Reinforcement learning use value functions whereas active inference uses priors to set objectives
        
        the pragmatic value will be computed in emodels and will be the basis for the first q value because 
        
        the pragmatic value deals with exploitation and selection based on realizing the objective set by prior
        
        note: observation noise is simply a counterpart of prior precision that allows us to adjust how much weight
        to put on observations as updates to the prior. in short we can make updates small by making the weight of 
        prior precision extremely large or making observation noise large. conversely 
        we can make the updates large by making prior precision extremely small or by making the obs noise small 
        
        we should note that prior precision acts as a gain control devide for amplifying signals. This may also be related
        to attention which is related to consciousness ans learning. 
        
        optimization of precisions is the statstical-mechanical basis of signal prioritisation in general
        
        to do this we need to be able to predict precisons so that we can weight the expected precision of error signals
        over the precision of outgoing predictions
        
        optimization beyond action and perception, the adjustment of precision to match the ampltitude of incoming
        prediction errors is the basis of consciousness in fristons work 
        
        the next phase of pegasus will be the following: for learning we need to do the inverse of inference 
        
        which involves optimizing beliefs about relationships between variables in the generative model. This may only
        occur after states have been inferred 
        
        to proceed with the next phase we will have to dispense with the mixer network 
        
        we should use the memory example rather then the election example as our primary reference mainly due to the fact
        that we are not using two latent variables 
        
        we dont need tow wory about sample size or optimizer since here we are optimizing over psuedo observations
        
        at some point tasha would like to use this project to model the brain of a dog as well as a human
        
        tasha would like visualization for simulated nueurons 
        
        
        our final two options are:
        
        set explore setting to false. Do expected free energy optimization on emodels and jettison efe
        
        or
        
        keep efe. calculate only pragmatic value on emodels and do full expected free energy calculation on EFE
        
        or 
        
        keep efe. do full calculations of epistemic and pragmatic in emodels.  
          
        On may 15th 2023 let us try to cut out efe and do the full Expected free energy calculation in emodel 
        """
            
        guide = ESM()# marginal_guide = posterior predictive entropy d can also be expressed as pi 
        pyro.clear_param_store()
        # To reduce noise when comparing designs, we will use the precomputed value of H(p(w))
        # By passing eig=False, we tell Pyro not to estimate the prior entropy on each run
        # The return value of `posterior_eig` is then -E_p(w,y)[log q(w|y)] where w = x  
        with pyro.plate_stack("plate", l.shape[:-1]):
            theta = pyro.sample("theta", dist.Dirichlet(1))#Normal(0, 0.8))# in our case this will represent the reward
            # Share theta across the number of rounds of the experiment
            # This represents repeatedly testing the same participant
            theta = theta.unsqueeze(-1)
            # This define a *logistic regression* model for y
            #logit_p = sensitivity * (theta - l)
            # The event shape represents responses from the same participant
            y = pyro.sample("y", dist.Bernoulli(logits=logit_p).to_event(1))
            return y
            #ape = average posterior entropy
        #we dont need tow wory about sample size or optimizer since here we are optimizing over psuedo observations
        ape = posterior_eig(model, allocation, "y", "theta", 10, 12500, guide,
                        Adam({"lr": 0.001}), eig=False, final_num_samples=10000)
        
        eigs[strategy] = prior_entropy - ape
        print(eigs[strategy].item())

        if eigs[strategy] > best_eig:
            best_strategy, best_eig = strategy, eigs[strategy]

        self.fc1 = nn.Linear(self.obs_size, self.rnn_hidden_dim)
        self.rnn = nn.GRUCell(self.rnn_hidden_dim, self.rnn_hidden_dim)
        
        self.fc2 = nn.Linear(self.rnn_hidden_dim, num_outputs)
        self.n_agents = model_config["n_agents"]

    @override(ModelV2)
    def get_initial_state(self):

        return [
            self.fc1.weight.new(self.n_agents, self.rnn_hidden_dim).zero_().squeeze(0)
        ]

    @override(ModelV2)
    def forward(self,x,input_dict, hidden_state, seq_lens):

        #negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)
                # Place hidden states on same device as model.
        def utterance_prior():
            ix = pyro.sample("utt", dist.Categorical(probs=torch.ones(3) / 3))
            return ["none","some","all"][ix]
        @Marginal
        def speaker():
            alpha = 1.
            with poutine.scale(scale=torch.tensor(alpha)):
                state = utterance_prior()
                pyro.sample("POMDP",state , obs=input_dict["obs_flat"])
            return utterance
        guide = ESM()

        # Step 3: learn the posterior using all data seen so far
        

        conditioned_model = pyro.condition(model, {"y": input_dict["obs_flat"]})
        svi = SVI(conditioned_model,
                  guide,
                  scheduler,#Adam({"lr": .005}),
                  loss=Trace_ELBO(),
                  num_samples=1)
        #num_iters = 2000
        #for i in range(num_iters):
        elbo = svi.step(ls)
        
        history.append((pyro.param("posterior_mean").detach().clone().numpy(),
                        pyro.param("posterior_sd").detach().clone().numpy()))
        current_model = make_model(pyro.param("posterior_mean").detach().clone(),
                                   pyro.param("posterior_sd").detach().clone())
        print("Estimate of \u03b8: {:.3f} \u00b1 {:.3f}\n".format(*history[-1]))     
                                
                                
        utterance = speaker()
        q = elbo - utterance 
        #####################################################################################
        x = nn.functional.relu(self.fc1(input_dict["obs_flat"].float()))
        h_in = hidden_state[0].reshape(-1, self.rnn_hidden_dim)
        h = self.rnn(x, h_in)
        """
        
        model = gpflow.models.VGP((x, h_in),kernel=gpflow.kernels.SquaredExponential(),
        likelihood=gpflow.likelihoods.Bernoulli())
        opt = gpflow.optimizers.Scipy()
        opt.minimize(model.training_loss, model.trainable_variables)
        model = gpflow.models.SVGP(
        kernel=gpflow.kernels.SquaredExponential(),
        likelihood=gpflow.likelihoods.Bernoulli(),
        inducing_variable=np.linspace(0.0, 1.0, 4)[:, None],)
        opt = gpflow.optimizers.Scipy()
        opt.minimize(model.training_loss_closure((X, Y)), model.trainable_variables)
        gpflow.utilities.print_summary(model, "notebook")


        
        """
        #q = self.fc2(h)
        #model = tfk.Model(inputs=i(input_dict["obs_flat"]),outputs=z(h))
        #i = tfkl.InputLayer(input_shape=input_shape)
        #x = tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),activation=None)(i)
        """
        prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),reinterpreted_batch_ndims=1)
        x  = tf.keras.layers.Dense(50,kernel_initializer='ones', use_bias=False,kernel_regularizer=tfpl.KLDivergenceRegularizer(prior))(h)
        #x = tfpl.MultivariateNormalTriL(d)
        z = tfp.layers.VariationalGaussianProcess(
                num_inducing_points=num_inducing_points, kernel_provider=RBFKernelFn(dtype=x.dtype) , event_shape=(1,),
                inducing_index_points_initializer=tf.compat.v1.constant_initializer(
                    np.linspace(0,x_range, num=1125,
                                dtype=x.dtype)[..., np.newaxis]), unconstrained_observation_noise_variance_initializer=(tf.compat.v1.constant_initializer(np.log(np.expm1(1.)).astype(x.dtype))),variational_inducing_observations_scale_initializer=(tf.compat.v1.constant_initializer(np.log(np.expm1(1.)).astype(x.dtype))), mean_fn=None,
                jitter=1e-06, convert_to_tensor_fn=tfp.distributions.Distribution.sample)(x)

        model = tfk.Model(inputs=x,outputs=q)#decoder(input_dict["obs_flat"][0]))
        
        yhat = model(x_tst)#Note that this is a distribution not a tensor
        num_samples = 13 
        for i in range(num_samples):
            sample_ = yhat.sample().numpy()
            sample_[..., 0].T
            e = sample_[..., 0].T
            r.append(e)
  
        rfinal = (r[0]+r[1]+r[2]+r[3]+r[4]+r[5]+r[6]+r[7]+r[8]+r[9]+r[10]+r[11]+r[12])/13
        
        x = nn.functional.relu(self.fc1(input_dict["obs_flat"].float()))
        #
        h_in = hidden_state[0].reshape(-1, self.rnn_hidden_dim)
        rnn = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(4))
        
        h = self.rnn(x,h_in)
        q = self.fc2(h)
        """
        #negloglik = lambda x, rv_x: -rv_x.log_prob(h)#tf.keras.losses.KLDivergence()
        #model.add_loss(negloglik)
        return q, [h]

def _get_size(obs_space):
    return get_preprocessor(obs_space)(obs_space).size
ivy.set_framework('torch')

In [13]:
torch, nn = try_import_torch()
#@ti.func
@accelerate_model()
class bmodel(TorchModelV2, nn.Module,ivy.Module):
    """The default RNN model for QMIX."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):#(self, obs_space, action_space, num_outputs, model_config):
        TorchModelV2.__init__(
             self, obs_space, action_space, num_outputs, model_config, name#self, obs_space, action_space, num_outputs, model_config
        )
        nn.Module.__init__(self)
        self.obs_size = _get_size(obs_space)
        self.rnn_hidden_dim = model_config["lstm_cell_size"]
        tfk = tf.keras
        tfkl = tf.keras.layers
        input_shape=2
        tfpl = tfp.layers
        tfd = tfp.distributions

        tfd = tfp.distributions
        encoded_size = 16
        
        """
        pseudo code
        
        assign a prior to be the posterior of the output of previous agent
        
        if no output of agent is found we have a preset prior. Prior is going to be a dirchlet
        
        #under advice from Tasha and Dimitrov and given what we know about
        #long term dependencce int he brain we are using LSTMs for neuronal agents
        
        
        """
        prior = tfp.distributions.Dirichlet(concentration)
        prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                            reinterpreted_batch_ndims=1)
        tfpl = tfp.layers
        
        """
        encoder = tfk.Sequential([
            tfkl.InputLayer(input_shape=input_shape),
            #tfkl.Dense(8)
            tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
                          activation=None),

            tfpl.MultivariateNormalTriL(
                    encoded_size,
                    activity_regularizer=tfpl.KLDivergenceRegularizer(priora, weight=1.2)),
        ])
        decoder = tfk.Sequential([
            tfkl.InputLayer(input_shape=[encoded_size]),
            tfkl.Dense(tfpl.IndependentNormal.params_size(encoded_size))
            #tfpl.IndependentBernoulli(input_shape, tfd.Bernoulli.logits)

        ])
        """
        
        self.fc1 = nn.Linear(self.obs_size, self.rnn_hidden_dim)
        self.rnn = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(4))#nn.GRUCell(self.rnn_hidden_dim, self.rnn_hidden_dim)
        self.fc2 = nn.Linear(self.rnn_hidden_dim, num_outputs)
        self.n_agents = model_config["n_agents"]

    @override(ModelV2)
    def get_initial_state(self):

        return [
            self.fc1.weight.new(self.n_agents, self.rnn_hidden_dim).zero_().squeeze(0)
        ]

    @override(ModelV2)
    def forward(self,x,input_dict, hidden_state, seq_lens):

        #negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)
                # Place hidden states on same device as model.
        
        x = nn.functional.relu(self.fc1(input_dict["obs_flat"].float()))
        #
        h_in = hidden_state[0].reshape(-1, self.rnn_hidden_dim)
        rnn = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(4))
        
        h = self.rnn(x,h_in)
        q = self.fc2(h)

        """
        vae = tfk.Model(inputs=encoder(input_dict["obs_flat"]),
                outputs=decoder(input_dict["obs_flat"][0]))
        """

        return q, [h]

def _get_size(obs_space):
    return get_preprocessor(obs_space)(obs_space).size

In [15]:
import tensorflow as tf
import numpy as np
y = np.ones(30)
y = np.expand_dims(y,1)
x = tf.keras.layers.Dense(1)(y)

In [14]:
from ray.rllib.utils.framework import try_import_torch

torch, nn = try_import_torch()

"""
next objective for april 8th 2023: we are going to have to modify the mixer network 

The mixer network is going to be a bayesian tensorflow network with a -ELBO builtin loss and a kullbacker leibeler loss
that will be seen in the QMIX loss function

We will have a configurable parameter that depending on whether the policy is high or low will have either -elbo or 
kullbacker leibler as builtin losses with a generic bayesian neural network . 

finally we will have emodel which will have a neural network guide

"""


#@ti.func
#@ti.data_oriented
class QMixer(nn.Module):
    def __init__(self, n_agents, state_shape, mixing_embed_dim):
        super(QMixer, self).__init__()

        self.n_agents = n_agents
        self.embed_dim = mixing_embed_dim
        self.state_dim = int(np.prod(state_shape))

        self.hyper_w_1 = nn.Linear(self.state_dim, self.embed_dim * self.n_agents)#tfkl.Embedding(self.state_dim,  self.embed_dim * self.n_agents)#
        self.hyper_w_final = nn.Linear(self.state_dim, self.embed_dim)#tfkl.Embedding(self.state_dim, self.embed_dim)

        # State dependent bias for hidden layer
        self.hyper_b_1 = nn.Linear(self.state_dim, self.embed_dim)#tfkl.Embedding(self.state_dim, self.embed_dim)

        # V(s) instead of a bias for the last layers
        self.V = nn.Sequential(
            nn.Linear(self.state_dim, self.embed_dim),#tfkl.Embedding(self.state_dim, self.embed_dim)
            nn.ReLU(),
            # tf.nn.relu
            nn.Linear(self.embed_dim, 1),#tfkl.Embedding(self.embed_dim, 1)
        )
        
    def forward(self, agent_qs, states):
        """Forward pass for the mixer.
        Args:
            agent_qs: Tensor of shape [B, T, n_agents, n_actions]
            states: Tensor of shape [B, T, state_dim]
        """
        """
        
        nn.Sequential(
            torch.abs(self.hyper_w_1(states))
            self.hyper_b_1(states)
            nn.functional.elu(torch.bmm(agent_qs, w1) + b1)
            torch.abs(self.hyper_w_final(states))
            self.V
            
        )
        tfkl.InputLayer(self.state_dim,self.embed_dim*self.n_agents)
        
        self.hyper_w_1 = tf.keras.activations.linear(self.state_dim, self.embed_dim * self.n_agents)
        
        tfkl.Embedding(1000, 64)#, input_length=10)
        
        x.numpy()
        
        #batch matrix multiplication
        tf.matmul()
        
        model = keras.Model(inputs=[states, agent_qs, tags_input], outputs=output
        
        
        #################################################################################################
        w1 = tf.math.abs(tfkl.Embedding(self.state_dim,  self.embed_dim * self.n_agents)(states))
        b1 = tfkl.Embedding(self.state_dim, self.embed_dim)(states)
        w1 = tf.reshape(w1, [-1, self.n_agents, self.embed_dim])
        b1 = tf.reshape(b1,[-1, 1, self.embed_dim])
        hidden = tf.nn.elu(tf.matmul(agent_qs, w1)+ b1)
        w_final = tf.math.abs(tfkl.Embedding(self.state_dim, self.embed_dim)(states))
        w_final = tf.reshape(-1, self.embed_dim, 1)
        v =  tfkl.Embedding(self.V(states),-1, 1, 1)
        y = tf.matmul(hidden, w_final) + v
        q_tot = tf.reshape(y,[bs, -1, 1])
        
        """
        
        bs = agent_qs.size(0)
        states = states.reshape(-1, self.state_dim)#tf.keras.layers.Reshape((3, 4), input_shape=(12,))
        agent_qs = agent_qs.view(-1, 1, self.n_agents)
        # First layer
        
        w1 = torch.abs(self.hyper_w_1(states))
        
        b1 = self.hyper_b_1(states)
        w1 = w1.view(-1, self.n_agents, self.embed_dim)
        b1 = b1.view(-1, 1, self.embed_dim)
        hidden = nn.functional.elu(torch.bmm(agent_qs, w1) + b1)#tf.nn.elu(tf.matmul(agent_qs, w1)+ b1)
        # Second layer #tf.nn.elu()
        w_final = torch.abs(self.hyper_w_final(states))
        w_final = w_final.view(-1, self.embed_dim, 1)
        # State-dependent bias
        v = self.V(states).view(-1, 1, 1)
        # Compute final output
        y = torch.bmm(hidden, w_final) + v#tf.matmul(hidden, w_final)
        
        # Reshape and return
        q_tot = y.view(bs, -1, 1)
        return q_tot

In [15]:
#@ti.data_oriented
class QMixLoss(nn.Module):
    def __init__(
        self,
        model,
        target_model,
        mixer,
        target_mixer,
        n_agents,
        n_actions,
        double_q=True,
        gamma=0.99,
    ):
        nn.Module.__init__(self)
        self.model = model
        self.target_model = target_model
        self.mixer = mixer
        self.target_mixer = target_mixer
        self.n_agents = n_agents
        self.n_actions = n_actions
        self.double_q = double_q
        self.gamma = gamma

    def forward(
        self,
        rewards,
        actions,
        terminated,
        mask,
        obs,
        next_obs,
        action_mask,
        next_action_mask,
        state=None,
        next_state=None,
    ):
        """Forward pass of the loss.
        Args:
            rewards: Tensor of shape [B, T, n_agents]
            actions: Tensor of shape [B, T, n_agents]
            terminated: Tensor of shape [B, T, n_agents]
            mask: Tensor of shape [B, T, n_agents]
            obs: Tensor of shape [B, T, n_agents, obs_size]
            next_obs: Tensor of shape [B, T, n_agents, obs_size]
            action_mask: Tensor of shape [B, T, n_agents, n_actions]
            next_action_mask: Tensor of shape [B, T, n_agents, n_actions]
            state: Tensor of shape [B, T, state_dim] (optional)
            next_state: Tensor of shape [B, T, state_dim] (optional)
        """

        # Assert either none or both of state and next_state are given
        if state is None and next_state is None:
            state = obs  # default to state being all agents' observations
            next_state = next_obs
        elif (state is None) != (next_state is None):
            raise ValueError(
                "Expected either neither or both of `state` and "
                "`next_state` to be given. Got: "
                "\n`state` = {}\n`next_state` = {}".format(state, next_state)
            )

        # Calculate estimated Q-Values
        mac_out = _unroll_mac(self.model, obs)

        # Pick the Q-Values for the actions taken -> [B * n_agents, T]
        chosen_action_qvals = torch.gather(
            mac_out, dim=3, index=actions.unsqueeze(3)
        ).squeeze(3)

        # Calculate the Q-Values necessary for the target
        target_mac_out = _unroll_mac(self.target_model, next_obs)

        # Mask out unavailable actions for the t+1 step
        ignore_action_tp1 = (next_action_mask == 0) & (mask == 1).unsqueeze(-1)
        target_mac_out[ignore_action_tp1] = -np.inf

        # Max over target Q-Values
        if self.double_q:
            # Double Q learning computes the target Q values by selecting the
            # t+1 timestep action according to the "policy" neural network and
            # then estimating the Q-value of that action with the "target"
            # neural network
            
            #target neural network does expected free energy while policy
            #neural network will be variational free energy

            # Compute the t+1 Q-values to be used in action selection
            # using next_obs
            mac_out_tp1 = _unroll_mac(self.model, next_obs)

            # mask out unallowed actions
            mac_out_tp1[ignore_action_tp1] = -np.inf

            # obtain best actions at t+1 according to policy NN
            cur_max_actions = mac_out_tp1.argmax(dim=3, keepdim=True)

            # use the target network to estimate the Q-values of policy
            # network's selected actions
            target_max_qvals = torch.gather(target_mac_out, 3, cur_max_actions).squeeze(
                3
            )
        else:
            target_max_qvals = target_mac_out.max(dim=3)[0]

        assert (
            target_max_qvals.min().item() != -np.inf
        ), "target_max_qvals contains a masked action; \
            there may be a state with no valid actions."

        # Mix
        if self.mixer is not None:
            chosen_action_qvals = self.mixer(chosen_action_qvals, state)
            target_max_qvals = self.target_mixer(target_max_qvals, next_state)

        # Calculate 1-step Q-Learning targets
        targets = rewards + self.gamma * (1 - terminated) * target_max_qvals
        """
        
        guide = pyro.infer.autoguide.AutoNormal(model)

        elbo = elbo_(model, guide)
        
        el = elbo(data)
        
        #rewards = {self.agent_1:kullbacker, self.agent_2: el}
        #loss = lambda y, rv_y: rv_y.variational_loss(y, kl_weight=np.array(batch_size, x.dtype) / x.shape[0])
                
        
        """
        
        # Td-error
        #we need to replace this with a variational free energy error
        td_error = chosen_action_qvals - targets.detach()
        te_error= tf.keras.losses.KLDivergence(chosen_action_qvals - targets.detach()).numpy()
        #negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)#this is ELBO 
        
        #we are going to ahve a kldivergence loss and a -ELBO regularizer for the mixer network 
        
        mask = mask.expand_as(te_error)

        # 0-out the targets that came from padded data
        masked_td_error = te_error * mask

        # Normal L2 loss, take mean over actual data
        
        #guide = pyro.infer.autoguide.AutoNormal(model)

        #elbo_ = pyro.infer.Trace_ELBO(num_particles=1)

        # Fix the model/guide pair
        #elbo = elbo_(model, guide)        
        
        #data = obs + preds 
        #loss = -ln()
        #loss = lambda y, rv_y: -rv_y.log_prob(y)
        loss = masked_error#(masked_td_error**2).sum() / mask.sum()
        return loss, mask, masked_td_error, chosen_action_qvals, targets

    
#this part just above is what we need to revise
    
#@ti.func
#@ti.data_oriented
class QMixTorchPolicy(TorchPolicy):
    """QMix impl. Assumes homogeneous agents for now.
    You must use MultiAgentEnv.with_agent_groups() to group agents
    together for QMix. This creates the proper Tuple obs/action spaces and
    populates the '_group_rewards' info field.
    Action masking: to specify an action mask for individual agents, use a
    dict space with an action_mask key, e.g. {"obs": ob, "action_mask": mask}.
    The mask space must be `Box(0, 1, (n_actions,))`.
    """

    def __init__(self, obs_space, action_space, config):
        # We want to error out on instantiation and not on import, because tune
        # imports all RLlib algorithms when registering them
        # TODO (Artur): Find a way to only import algorithms when needed
        if not torch:
            raise ImportError("Could not import PyTorch, which QMix requires.")

        _validate(obs_space, action_space)
        config = dict(ray.rllib.algorithms.qmix.qmix.DEFAULT_CONFIG, **config)
        self.framework = "torch"

        self.n_agents = 3000#len(obs_space.original_space.spaces)
        config["model"]["n_agents"] = self.n_agents
        self.n_actions = action_space.spaces[0].n
        self.h_size = config["model"]["lstm_cell_size"]
        self.has_env_global_state = False
        self.has_action_mask = False

        agent_obs_space = obs_space.original_space.spaces[0]
        if isinstance(agent_obs_space, gym.spaces.Dict):
            space_keys = set(agent_obs_space.spaces.keys())
            if "obs" not in space_keys:
                raise ValueError("Dict obs space must have subspace labeled `obs`")
            self.obs_size = _get_size(agent_obs_space.spaces["obs"])
            if "action_mask" in space_keys:
                mask_shape = tuple(agent_obs_space.spaces["action_mask"].shape)
                if mask_shape != (self.n_actions,):
                    raise ValueError(
                        "Action mask shape must be {}, got {}".format(
                            (self.n_actions,), mask_shape
                        )
                    )
                self.has_action_mask = True
            if ENV_STATE in space_keys:
                self.env_global_state_shape = _get_size(
                    agent_obs_space.spaces[ENV_STATE]
                )
                self.has_env_global_state = True
            else:
                self.env_global_state_shape = (self.obs_size, self.n_agents)
            # The real agent obs space is nested inside the dict
            config["model"]["full_obs_space"] = agent_obs_space
            agent_obs_space = agent_obs_space.spaces["obs"]
        else:
            self.obs_size = _get_size(agent_obs_space)
            self.env_global_state_shape = (self.obs_size, self.n_agents)
        #model = bmodel()#CModel()#CModel()
        #bmodel = Laplace(model,'regression', subset_of_weights='last_layer', hessian_structure='full')#
        ivy.set_framework('torch')
        #model = Laplace(model,'regression', subset_of_weights='last_layer', hessian_structure='full')
        bmodel = bmodel()
        emodel = emodel()
        bmodel = FullLaplace(bmodel,'regression',prior_precision=0.00000000000000000000001)#0.00000000000000000000001)#10000000000000000000000000)
        self.model = ModelCatalog.get_model_v2(
            agent_obs_space,
            action_space.spaces[0],
            self.n_actions,
            config["model"],
            framework="torch",
            name="model",
            default_model=emodel#a#RNNModel#bmodel()#RNNModel,
        )

        super().__init__(obs_space, action_space, config, model=self.model)

        self.target_model = ModelCatalog.get_model_v2(
            agent_obs_space,
            action_space.spaces[0],
            self.n_actions,
            config["model"],
            framework="torch",
            name="target_model",
            default_model=bmodel#bmodel()#RNNModel
        ).to(self.device)

        self.exploration = self._create_exploration()
        
        # Setup the mixer network.
        if config["mixer"] is None:
            self.mixer = None
            self.target_mixer = None
        elif config["mixer"] == "qmix":
            self.mixer = FullLaplace(QMixer(
                self.n_agents, self.env_global_state_shape, config["mixing_embed_dim"]
            ),prior_precision=1).to(self.device)
            self.target_mixer = FullLaplace(QMixer(
                self.n_agents, self.env_global_state_shape, config["mixing_embed_dim"]
            ),prior_precision=1).to(self.device)

        self.cur_epsilon = 1.0
        self.update_target()  # initial sync

        # Setup optimizer
        self.params = list(self.model.parameters())
        if self.mixer:
            self.params += list(self.mixer.parameters())
        self.loss = QMixLoss(
            self.model,
            self.target_model,
            self.mixer,
            self.target_mixer,
            self.n_agents,
            self.n_actions,
            self.config["double_q"],
            self.config["gamma"],
        )
        from torch.optim import RMSprop
        
        self.rmsprop_optimizer = RMSprop(
            params=self.params,
            lr=config["lr"],
            alpha=config["optim_alpha"],
            eps=config["optim_eps"],
        )#replace with dadptation 
        self.rs_prop_optimizer = DAdaptAdaGrad(
            params=self.params,
            alpha=config("optim_alpha"),
            eps = config["optim_eps"],
        
        )

    @override(TorchPolicy)
    def compute_actions_from_input_dict(
        self,
        input_dict: Dict[str, TensorType],
        explore: bool = None,
        timestep: Optional[int] = None,
        **kwargs,
    ) -> Tuple[TensorType, List[TensorType], Dict[str, TensorType]]:

        obs_batch = input_dict[SampleBatch.OBS]
        state_batches = []
        i = 0
        while f"state_in_{i}" in input_dict:
            state_batches.append(input_dict[f"state_in_{i}"])
            i += 1

        explore = explore if explore is not None else self.config["explore"]
        obs_batch, action_mask, _ = self._unpack_observation(obs_batch)
        # We need to ensure we do not use the env global state
        # to compute actions

        # Compute actions
        with torch.no_grad():
            q_values, hiddens = _mac(
                self.model,
                torch.as_tensor(obs_batch, dtype=torch.float, device=self.device),
                [
                    torch.as_tensor(np.array(s), dtype=torch.float, device=self.device)
                    for s in state_batches
                ],
            )
            avail = torch.as_tensor(action_mask, dtype=torch.float, device=self.device)
            masked_q_values = q_values.clone()
            masked_q_values[avail == 0.0] = -float("inf")
            masked_q_values_folded = torch.reshape(
                masked_q_values, [-1] + list(masked_q_values.shape)[2:]
            )
            actions, _ = self.exploration.get_exploration_action(
                action_distribution=TorchCategorical(masked_q_values_folded),
                timestep=timestep,
                explore=explore,
            )
            actions = (
                torch.reshape(actions, list(masked_q_values.shape)[:-1]).cpu().numpy()
            )
            hiddens = [s.cpu().numpy() for s in hiddens]

        return tuple(actions.transpose([1, 0])), hiddens, {}

    @override(TorchPolicy)
    def compute_actions(self, *args, **kwargs):
        return self.compute_actions_from_input_dict(*args, **kwargs)

    @override(TorchPolicy)
    def compute_log_likelihoods(
        self,
        actions,
        obs_batch,
        state_batches=None,
        prev_action_batch=None,
        prev_reward_batch=None,
    ):
        obs_batch, action_mask, _ = self._unpack_observation(obs_batch)
        return np.zeros(obs_batch.size()[0])

    @override(TorchPolicy)
    def learn_on_batch(self, samples):
        obs_batch, action_mask, env_global_state = self._unpack_observation(
            samples[SampleBatch.CUR_OBS]
        )
        (
            next_obs_batch,
            next_action_mask,
            next_env_global_state,
        ) = self._unpack_observation(samples[SampleBatch.NEXT_OBS])
        group_rewards = self._get_group_rewards(samples[SampleBatch.INFOS])

        input_list = [
            group_rewards,
            action_mask,
            next_action_mask,
            samples[SampleBatch.ACTIONS],
            samples[SampleBatch.TERMINATEDS],
            obs_batch,
            next_obs_batch,
        ]
        if self.has_env_global_state:
            input_list.extend([env_global_state, next_env_global_state])

        output_list, _, seq_lens = chop_into_sequences(
            episode_ids=samples[SampleBatch.EPS_ID],
            unroll_ids=samples[SampleBatch.UNROLL_ID],
            agent_indices=samples[SampleBatch.AGENT_INDEX],
            feature_columns=input_list,
            state_columns=[],  # RNN states not used here
            max_seq_len=self.config["model"]["max_seq_len"],
            dynamic_max=True,
        )
        # These will be padded to shape [B * T, ...]
        if self.has_env_global_state:
            (
                rew,
                action_mask,
                next_action_mask,
                act,
                terminateds,
                obs,
                next_obs,
                env_global_state,
                next_env_global_state,
            ) = output_list
        else:
            (
                rew,
                action_mask,
                next_action_mask,
                act,
                terminateds,
                obs,
                next_obs,
            ) = output_list
        B, T = len(seq_lens), max(seq_lens)

        def to_batches(arr, dtype):
            new_shape = [B, T] + list(arr.shape[1:])
            return torch.as_tensor(
                np.reshape(arr, new_shape), dtype=dtype, device=self.device
            )

        rewards = to_batches(rew, torch.float)
        actions = to_batches(act, torch.long)
        obs = to_batches(obs, torch.float).reshape([B, T, self.n_agents, self.obs_size])
        action_mask = to_batches(action_mask, torch.float)
        next_obs = to_batches(next_obs, torch.float).reshape(
            [B, T, self.n_agents, self.obs_size]
        )
        next_action_mask = to_batches(next_action_mask, torch.float)
        if self.has_env_global_state:
            env_global_state = to_batches(env_global_state, torch.float)
            next_env_global_state = to_batches(next_env_global_state, torch.float)

        # TODO(ekl) this treats group termination as individual termination
        terminated = (
            to_batches(terminateds, torch.float)
            .unsqueeze(2)
            .expand(B, T, self.n_agents)
        )

        # Create mask for where index is < unpadded sequence length
        filled = np.reshape(
            np.tile(np.arange(T, dtype=np.float32), B), [B, T]
        ) < np.expand_dims(seq_lens, 1)
        mask = (
            torch.as_tensor(filled, dtype=torch.float, device=self.device)
            .unsqueeze(2)
            .expand(B, T, self.n_agents)
        )

        # Compute loss
        loss_out, mask, masked_td_error, chosen_action_qvals, targets = self.loss(
            rewards,
            actions,
            terminated,
            mask,
            obs,
            next_obs,
            action_mask,
            next_action_mask,
            env_global_state,
            next_env_global_state,
        )

        # Optimise
        self.rsprop_optimizer.zero_grad()

        loss_out.backward()
        grad_norm_info = apply_grad_clipping(self, self.rsprop_optimizer, loss_out)
        self.rsprop_optimizer.step()

        mask_elems = mask.sum().item()
        stats = {
            "loss": loss_out.item(),
            "td_error_abs": masked_td_error.abs().sum().item() / mask_elems,
            "q_taken_mean": (chosen_action_qvals * mask).sum().item() / mask_elems,
            "target_mean": (targets * mask).sum().item() / mask_elems,
        }
        stats.update(grad_norm_info)

        return {LEARNER_STATS_KEY: stats}

    @override(TorchPolicy)
    def get_initial_state(self):  # initial RNN state
        return [
            s.expand([self.n_agents, -1]).cpu().numpy()
            for s in self.model.get_initial_state()
        ]

    @override(TorchPolicy)
    def get_weights(self):
        return {
            "model": self._cpu_dict(self.model.state_dict()),
            "target_model": self._cpu_dict(self.target_model.state_dict()),
            "mixer": self._cpu_dict(self.mixer.state_dict()) if self.mixer else None,
            "target_mixer": self._cpu_dict(self.target_mixer.state_dict())
            if self.mixer
            else None,
        }

    @override(TorchPolicy)
    def set_weights(self, weights):
        self.model.load_state_dict(self._device_dict(weights["model"]))
        self.target_model.load_state_dict(self._device_dict(weights["target_model"]))
        if weights["mixer"] is not None:
            self.mixer.load_state_dict(self._device_dict(weights["mixer"]))
            self.target_mixer.load_state_dict(
                self._device_dict(weights["target_mixer"])
            )

    @override(TorchPolicy)
    def get_state(self):
        state = self.get_weights()
        state["cur_epsilon"] = self.cur_epsilon
        return state

    @override(TorchPolicy)
    def set_state(self, state):
        self.set_weights(state)
        self.set_epsilon(state["cur_epsilon"])

    def update_target(self):
        self.target_model.load_state_dict(self.model.state_dict())
        if self.mixer is not None:
            self.target_mixer.load_state_dict(self.mixer.state_dict())
        logger.debug("Updated target networks")

    def set_epsilon(self, epsilon):
        self.cur_epsilon = epsilon

    def _get_group_rewards(self, info_batch):
        group_rewards = np.array(
            [info.get(GROUP_REWARDS, [0.0] * self.n_agents) for info in info_batch]
        )
        return group_rewards

    def _device_dict(self, state_dict):
        return {
            k: torch.as_tensor(v, device=self.device) for k, v in state_dict.items()
        }

    @staticmethod
    def _cpu_dict(state_dict):
        return {k: v.cpu().detach().numpy() for k, v in state_dict.items()}

    def _unpack_observation(self, obs_batch):
        """Unpacks the observation, action mask, and state (if present)
        from agent grouping.
        Returns:
            obs (np.ndarray): obs tensor of shape [B, n_agents, obs_size]
            mask (np.ndarray): action mask, if any
            state (np.ndarray or None): state tensor of shape [B, state_size]
                or None if it is not in the batch
        """

        unpacked = _unpack_obs(
            np.array(obs_batch, dtype=np.float32),
            self.observation_space.original_space,
            tensorlib=np,
        )

        if isinstance(unpacked[0], dict):
            assert "obs" in unpacked[0]
            unpacked_obs = [np.concatenate(tree.flatten(u["obs"]), 1) for u in unpacked]
        else:
            unpacked_obs = unpacked

        obs = np.concatenate(unpacked_obs, axis=1).reshape(
            [len(obs_batch), self.n_agents, self.obs_size]
        )

        if self.has_action_mask:
            action_mask = np.concatenate(
                [o["action_mask"] for o in unpacked], axis=1
            ).reshape([len(obs_batch), self.n_agents, self.n_actions])
        else:
            action_mask = np.ones(
                [len(obs_batch), self.n_agents, self.n_actions], dtype=np.float32
            )

        if self.has_env_global_state:
            state = np.concatenate(tree.flatten(unpacked[0][ENV_STATE]), 1)
        else:
            state = None
        return obs, action_mask, state

#@ti.func
def _validate(obs_space, action_space):
    if not hasattr(obs_space, "original_space") or not isinstance(
        obs_space.original_space, gym.spaces.Tuple
    ):
        raise ValueError(
            "Obs space must be a Tuple, got {}. Use ".format(obs_space)
            + "MultiAgentEnv.with_agent_groups() to group related "
            "agents for QMix."
        )
    if not isinstance(action_space, gym.spaces.Tuple):
        raise ValueError(
            "Action space must be a Tuple, got {}. ".format(action_space)
            + "Use MultiAgentEnv.with_agent_groups() to group related "
            "agents for QMix."
        )
    if not isinstance(action_space.spaces[0], gym.spaces.Discrete):
        raise ValueError(
            "QMix requires a discrete action space, got {}".format(
                action_space.spaces[0]
            )
        )
    if len({str(x) for x in obs_space.original_space.spaces}) > 1:
        raise ValueError(
            "Implementation limitation: observations of grouped agents "
            "must be homogeneous, got {}".format(obs_space.original_space.spaces)
        )
    if len({str(x) for x in action_space.spaces}) > 1:
        raise ValueError(
            "Implementation limitation: action space of grouped agents "
            "must be homogeneous, got {}".format(action_space.spaces)
        )

#@ti.func
def _mac(model, obs, h):
    """Forward pass of the multi-agent controller.
    Args:
        model: TorchModelV2 class
        obs: Tensor of shape [B, n_agents, obs_size]
        h: List of tensors of shape [B, n_agents, h_size]
    Returns:
        q_vals: Tensor of shape [B, n_agents, n_actions]
        h: Tensor of shape [B, n_agents, h_size]
    """
    B, n_agents = obs.size(0), obs.size(1)
    if not isinstance(obs, dict):
        obs = {"obs": obs}
    obs_agents_as_batches = {k: _drop_agent_dim(v) for k, v in obs.items()}
    h_flat = [s.reshape([B * n_agents, -1]) for s in h]
    q_flat, h_flat = model(obs_agents_as_batches, h_flat, None)
    return q_flat.reshape([B, n_agents, -1]), [
        s.reshape([B, n_agents, -1]) for s in h_flat
    ]
#@ti.func
def _unroll_mac(model, obs_tensor):
    """Computes the estimated Q values for an entire trajectory batch"""
    B = obs_tensor.size(0)
    T = obs_tensor.size(1)
    n_agents = obs_tensor.size(2)

    mac_out = []
    h = [s.expand([B, n_agents, -1]) for s in model.get_initial_state()]
    for t in range(T):
        q, h = _mac(model, obs_tensor[:, t], h)
        mac_out.append(q)
    mac_out = torch.stack(mac_out, dim=1)  # Concat over time

    return mac_out
#@ti.func
def _drop_agent_dim(T):
    shape = list(T.shape)
    B, n_agents = shape[0], shape[1]
    return T.reshape([B * n_agents] + shape[2:])
#@ti.func
def _add_agent_dim(T, n_agents):
    shape = list(T.shape)
    B = shape[0] // n_agents
    assert shape[0] % n_agents == 0
    return T.reshape([B, n_agents] + shape[1:])

In [16]:
#@ti.data_oriented
class TwoStepGame(MultiAgentEnv):

    def __init__(self, env_config):
        super().__init__()
        self.action_space = Discrete(2)
        self.state = None
        self.agent_1 = 0
        self.agent_2 = 1
        #self.agent_3=2
        self._skip_env_checking = True
        gpus_available = ray.get_gpu_ids()
        # MADDPG emits action logits instead of actual discrete actions
        self.actions_are_logits = env_config.get("actions_are_logits", False)
        self.one_hot_state_encoding = env_config.get("one_hot_state_encoding", False)
        self.with_state = env_config.get("separate_state_space", False)
        self._agent_ids = {0, 1}
        if not self.one_hot_state_encoding:
            self.observation_space = Tuple([Dict(
                    {
                        "obs": MultiDiscrete([2, 2, 2, 3]),
                        ENV_STATE: MultiDiscrete([2, 2, 2]),
                    }
                )])
            #self.observation_space = self.observation_space)
            self.with_state = False
        else:
            # Each agent gets the full state (one-hot encoding of which of the
            # three states are active) as input with the receiving agent's
            # ID (1 or 2) concatenated onto the end.
            if self.with_state:
                self.observation_space = Dict(
                    {
                        "obs": MultiDiscrete([2, 2, 2, 3]),
                        ENV_STATE: MultiDiscrete([2, 2, 2]),
                    }
                )
            else:
                self.observation_space = MultiDiscrete([2, 2, 2, 3])

    def reset(self, *, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)
        self.state = np.array([1, 0, 0])
        return self._obs(), {}

    def step(self, action_dict):

        state_index = np.flatnonzero(self.state)
        if state_index == 0:
            action = action_dict[self.agent_1]
            assert action in [0, 1], action
            if action == 0:
                self.state = np.array([0, 1, 0])
            else:
                self.state = np.array([0, 0, 1])
            global_rew = 0
            terminated = False
        elif state_index == 1:
            global_rew = 7
            terminated = True
        else:
            if action_dict[self.agent_1] == 0 and action_dict[self.agent_2] == 0:
                global_rew = 0
            elif action_dict[self.agent_1] == 1 and action_dict[self.agent_2] == 1:
                global_rew = 8
            else:
                global_rew = 1
            terminated = True
        
        rewards = {self.agent_1: global_rew / 2.0, self.agent_2: global_rew / 2.0}
        obs = self._obs()
        terminateds = {"__all__": terminated}
        truncateds = {"__all__": False}
        infos = {
            self.agent_1: {"done": terminateds["__all__"]},
            self.agent_2: {"done": terminateds["__all__"]},
            #self.agent_2: {"done": terminateds["__all__"]},
        }
        return obs, rewards, terminateds, truncateds, infos

    def _obs(self):
        if self.with_state:
            return {
                self.agent_1: {"obs": self.agent_1_obs(), ENV_STATE: self.state},
                self.agent_2: {"obs": self.agent_2_obs(), ENV_STATE: self.state},
                #self.agent_3: {"obs": self.agent_3_obs(), ENV_STATE: self.state},
            }
        else:
            return {self.agent_1: self.agent_1_obs(), self.agent_2: self.agent_2_obs()}

    def agent_1_obs(self):
        if self.one_hot_state_encoding:
            return np.concatenate([self.state, [1]])
        else:
            return np.flatnonzero(self.state)[0]

    def agent_2_obs(self):
        if self.one_hot_state_encoding:
            return np.concatenate([self.state, [2]])
        else:
            return np.flatnonzero(self.state)[0] + 3


        #if self.render_mode == "rgb_array":
            #return self._render_frame()

class TwoStepGameWithGroupedAgents(MultiAgentEnv):
    def __init__(self, env_config):
        super().__init__()
        env = TwoStepGame(env_config)
        tuple_obs_space = Tuple([env.observation_space, env.observation_space])#, env.observation_space,env.observation_space])
        tuple_act_space = Tuple([env.action_space, env.action_space])#, env.action_space,env.action_space])

        """
        self.env = env.with_agent_groups(
            groups={"agents": [0, 1,2]},
            obs_space=tuple_obs_space,
            act_space=tuple_act_space,
        )
        """
        self.env = env.with_agent_groups({
            "group1": ["agent1", "agent2"], # doctest: +SKIP
            #"group2": ["agent4", "agent5"], # doctest: +SKIP
            }) 
        self.observation_space = self.env.observation_space
        self.action_space = self.env.action_space
        #self._agent_ids = {"agents"}
        self._skip_env_checking = True

    def reset(self, *, seed=None, options=None):
        return self.env.reset(seed=seed, options=options)

    def step(self, actions):
        return self.env.step(actions)

In [17]:
import gymnasium as gym
from gymnasium.spaces import Box, Discrete, Tuple
import logging
import random

from ray.rllib.env import MultiAgentEnv

logger = logging.getLogger(__name__)


"""
note: QMIX is ONLY Discrete.VDN can also not be used without discrete

Should consider using a preprocessor to discretize the observation space. 

For action space we will need to have a lowest level discrete space of 100 

"""
class Pegasus(MultiAgentEnv):
    def __init__(self, env_config):
        super().__init__()
        self.action_space = Discrete(2)
        self.state = None
        self.agent_1 = 0
        self.agent_2 = 1
        self._skip_env_checking = True
        gpus_available = ray.get_gpu_ids()
        # MADDPG emits action logits instead of actual discrete actions
        self.actions_are_logits = env_config.get("actions_are_logits", False)
        self.one_hot_state_encoding = env_config.get("one_hot_state_encoding", False)
        self.with_state = env_config.get("separate_state_space", False)
        self._agent_ids = {0, 1}
        if not self.one_hot_state_encoding:
            self.observation_space = Tuple([Dict(
                    {
                        "obs": MultiDiscrete([2, 2, 2, 3]),
                        ENV_STATE: MultiDiscrete([2, 2, 2]),
                    }
                )])
            #self.observation_space = self.observation_space)
            self.with_state = False
        else:
            # Each agent gets the full state (one-hot encoding of which of the
            # three states are active) as input with the receiving agent's
            # ID (1 or 2) concatenated onto the end.
            if self.with_state:
                self.observation_space = Dict(
                    {
                        "obs": MultiDiscrete([2, 2, 2, 3]),
                        ENV_STATE: MultiDiscrete([2, 2, 2]),
                    }
                )
            else:
                self.observation_space = MultiDiscrete([2, 2, 2, 3])
    def reset(self, *, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)
        self.state = np.array([1, 0, 0])
        return self._obs(), {}
    def step(self, action_dict):

        state_index = np.flatnonzero(self.state)
        if state_index == 0:
            action = action_dict[self.agent_1]
            assert action in [0, 1], action
            if action == 0:
                self.state = np.array([0, 1, 0])
            else:
                self.state = np.array([0, 0, 1])
            global_rew = 0
            terminated = False
        elif state_index == 1:
            global_rew = 7
            terminated = True
        else:
            if action_dict[self.agent_1] == 0 and action_dict[self.agent_2] == 0:
                global_rew = 0
            elif action_dict[self.agent_1] == 1 and action_dict[self.agent_2] == 1:
                global_rew = 8
            else:
                global_rew = 1
            terminated = True
        
        rewards = {self.agent_1: global_rew / 2.0, self.agent_2: global_rew / 2.0}
        obs = self._obs()
        terminateds = {"__all__": terminated}
        truncateds = {"__all__": False}
        infos = {
            self.agent_1: {"done": terminateds["__all__"]},
            self.agent_2: {"done": terminateds["__all__"]},
            #self.agent_2: {"done": terminateds["__all__"]},
        }
        return obs, rewards, terminateds, truncateds, infos
    def _obs(self):
        if self.with_state:
            return {
                self.agent_1: {"obs": self.agent_1_obs(), ENV_STATE: self.state},
                self.agent_2: {"obs": self.agent_2_obs(), ENV_STATE: self.state},
                #self.agent_3: {"obs": self.agent_3_obs(), ENV_STATE: self.state},
            }
        else:
            return {self.agent_1: self.agent_1_obs(), self.agent_2: self.agent_2_obs()}

    def agent_1_obs(self):
        if self.one_hot_state_encoding:
            return np.concatenate([self.state, [1]])
        else:
            return np.flatnonzero(self.state)[0]

    def agent_2_obs(self):
        if self.one_hot_state_encoding:
            return np.concatenate([self.state, [2]])
        else:
            return np.flatnonzero(self.state)[0] + 3
    
class HierarchicalGroupedPegasus(MultiAgentEnv):
    def __init__(self, env_config):
        super().__init__()
        self.flat_env = Pegasus(env_config)
        #env = TwoStepGame(env_config)


        self.observation_space = self.env.observation_space
        self.action_space = self.env.action_space
        self._agent_ids = {"agents"}
        self._skip_env_checking = True

        self.env = env.with_agent_groups(
            groups={"agents": [0, 1]},
            obs_space=tuple_obs_space,
            act_space=tuple_act_space,
        )
        tuple_obs_space = Tuple([env.observation_space, env.observation_space])
        tuple_act_space = Tuple([env.action_space, env.action_space])
    def reset(self, *, seed=None, options=None):
        if seed is not None:
            np.random.seed(seed)
        self.state = np.array([1, 0, 0])
        return self._obs(), {}
    def _high_level_step(self, action):
        self.current_goal = action
        obs = {self.low_level_agent_id: [self.cur_obs, self.current_goal]}
        rew = {self.low_level_agent_id: 0}
        done = truncated = {"__all__": False}
        return obs, rew, done, truncated, {}
    def _low_level_step(self, action):
        logger.debug("Low level agent step {}".format(action))
        self.steps_remaining_at_level -= 1
        cur_pos = tuple(self.cur_obs[0])
        goal_pos = self.flat_env._get_new_pos(cur_pos, self.current_goal)

        # Step in the actual env
        f_obs, f_rew, f_terminated, f_truncated, info = self.flat_env.step(action)
        new_pos = tuple(f_obs[0])
        self.cur_obs = f_obs
        # Calculate low-level agent observation and reward
        obs = {self.low_level_agent_id: [f_obs, self.current_goal]}
        if new_pos != cur_pos:
            if new_pos == goal_pos:
                rew = {self.low_level_agent_id: 1}
            else:
                rew = {self.low_level_agent_id: -1}
        else:
            rew = {self.low_level_agent_id: 0}

        # Handle env termination & transitions back to higher level.
        terminated = {"__all__": False}
        truncated = {"__all__": False}
        if f_terminated or f_truncated:
            terminated["__all__"] = f_terminated
            truncated["__all__"] = f_truncated
            logger.debug("high level final reward {}".format(f_rew))
            rew["high_level_agent"] = f_rew
            obs["high_level_agent"] = f_obs
        elif self.steps_remaining_at_level == 0:
            terminated[self.low_level_agent_id] = True
            truncated[self.low_level_agent_id] = False
            rew["high_level_agent"] = 0
            obs["high_level_agent"] = f_obs

        return obs, rew, terminated, truncated, {self.low_level_agent_id: info}

# Agent has to traverse the maze from the starting position S -> F
# Observation space [x_pos, y_pos, wind_direction]
# Action space: stay still OR move in current wind direction
#env = HierarchicalGroupedPegasus()
#enva = Pegasus()

In [17]:
from gymnasium.spaces import Discrete, MultiDiscrete, Space
from typing import Union, Optional

from ray.rllib.utils.annotations import PublicAPI
from ray.rllib.models.action_dist import ActionDistribution
from ray.rllib.models.tf.tf_action_dist import Categorical
from ray.rllib.models.torch.torch_action_dist import TorchCategorical
from ray.rllib.utils.annotations import override
from ray.rllib.utils.exploration.stochastic_sampling import StochasticSampling
from ray.rllib.utils.framework import TensorType

@PublicAPI
class EFE(StochasticSampling):
    """Special case of StochasticSampling w/ Categorical and temperature param.
    Returns a stochastic sample from a Categorical parameterized by the model
    output divided by the temperature. Returns the argmax iff explore=False.
    
    In our case we will be this is where we will create the pragmatic value and together with the epistemic value
    
    
    """

    def __init__(
        self,
        action_space: Space,
        *,
        framework: Optional[str],
        temperature: float = 1.0,
        **kwargs
    ):
        """Initializes a EFE Exploration object.
        Args:
            action_space: The gym action space used by the environment.
            temperature: The temperature to divide model outputs by
                before creating the Categorical distribution to sample from.
            framework: One of None, "tf", "torch".
        """
        assert isinstance(action_space, (Discrete, MultiDiscrete))
        super().__init__(action_space, framework=framework, **kwargs)
        self.temperature = temperature

    @override(StochasticSampling)
    def get_exploration_action(
        self,
        action_distribution: ActionDistribution,
        timestep: Union[int, TensorType],
        explore: bool = True,
    ):
        #pragmatic value function 

        @Marginal
        def speaker(state):
            alpha = 1.
            with poutine.scale(scale=torch.tensor(alpha)):
                utterance = utterance_prior()
                pyro.sample("listener", literal_listener(utterance), obs=state)
            return utterance
        cls = type(action_distribution)
        assert issubclass(cls, (Categorical, TorchCategorical))
        # Re-create the action distribution with the correct temperature
        # applied.
        dist = cls(action_distribution.inputs, self.model, temperature=self.temperature)
        # Step 3: learn the posterior using all data seen so far
        guide = ESM()
        model = self.model
        conditioned_model = pyro.condition(model, {"y": action_distribution.inputs})
        svi = SVI(conditioned_model,
                  guide,
                  Adam({"lr": .005}),
                  loss=Trace_ELBO(),
                  num_samples=100)
        #num_iters = 2000
        #for i in range(num_iters):
        elbo = svi.step(ls)
        
        history.append((pyro.param("posterior_mean").detach().clone().numpy(),
                        pyro.param("posterior_sd").detach().clone().numpy()))
        current_model = make_model(pyro.param("posterior_mean").detach().clone(),
                                   pyro.param("posterior_sd").detach().clone())
        print("Estimate of \u03b8: {:.3f} \u00b1 {:.3f}\n".format(*history[-1]))

        """
        
        for pragmatic value we can use soem of the math for Rational Speech Act framework
        
        note: "allocation" is used together with alpha to create a distribution which is the variable y. 
        
        eig = marginal_eig(model,
                           candidate_designs,       # design, or in this case, tensor of possible designs
                           "y",                     # site label of observations, could be a list
                           "theta",                 # site label of 'targets' (latent variables), could also be list
                           num_samples=100,         # number of samples to draw per step in the expectation
                           num_steps=num_steps,     # number of gradient steps
                           guide=marginal_guide,    # guide q(y)
                           optim=optimizer,         # optimizer with learning rate decay
                           final_num_samples=10000  # at the last step, we draw more samples
                                                    # for a more accurate EIG estimate
                          )
                          
        we should use pragmatic for emodels and complete either full or last half or first half of epistemic for efe 
        """

        #self.model  is the first agent model we use to get the first q value used for model selection
        # Delegate to super method.
        return super().get_exploration_action(
            action_distribution=dist, timestep=timestep, explore=explore)

In [18]:
#our custom replacement for softq in explore config
from gymnasium.spaces import Discrete, MultiDiscrete, Space
from typing import Union, Optional

from ray.rllib.utils.annotations import PublicAPI
from ray.rllib.models.action_dist import ActionDistribution
from ray.rllib.models.tf.tf_action_dist import Categorical
from ray.rllib.models.torch.torch_action_dist import TorchCategorical
from ray.rllib.utils.annotations import override
from ray.rllib.utils.exploration.stochastic_sampling import StochasticSampling
from ray.rllib.utils.framework import TensorType

@PublicAPI
class SoftQ(StochasticSampling):
    """Special case of StochasticSampling w/ Categorical and temperature param.
    Returns a stochastic sample from a Categorical parameterized by the model
    output divided by the temperature. Returns the argmax iff explore=False.
    """

    def __init__(
        self,
        action_space: Space,
        *,
        framework: Optional[str],
        temperature: float = 1.0,
        **kwargs
    ):
        """Initializes a SoftQ Exploration object.
        Args:
            action_space: The gym action space used by the environment.
            temperature: The temperature to divide model outputs by
                before creating the Categorical distribution to sample from.
            framework: One of None, "tf", "torch".
        """
        assert isinstance(action_space, (Discrete, MultiDiscrete))
        super().__init__(action_space, framework=framework, **kwargs)
        self.temperature = temperature

    @override(StochasticSampling)
    def get_exploration_action(
        self,
        action_distribution: ActionDistribution,
        timestep: Union[int, TensorType],
        explore: bool = True,
    ):
        cls = type(action_distribution)
        assert issubclass(cls, (Categorical, TorchCategorical))
        # Re-create the action distribution with the correct temperature
        # applied.
        dist = cls(action_distribution.inputs, self.model, temperature=self.temperature)
        # Delegate to super method.
        return super().get_exploration_action(
            action_distribution=dist, timestep=timestep, explore=explore
        )

In [18]:
from typing import Optional, Type

from ray.rllib.algorithms.algorithm_config import AlgorithmConfig, NotProvided
from ray.rllib.algorithms.simple_q.simple_q import SimpleQ, SimpleQConfig
#from ray.rllib.algorithms.qmix.qmix_policy import QMixTorchPolicy
from ray.rllib.utils.replay_buffers.utils import update_priorities_in_replay_buffer
from ray.rllib.execution.rollout_ops import (
    synchronous_parallel_sample,
)
from ray.rllib.execution.train_ops import (
    multi_gpu_train_one_step,
    train_one_step,
)
from ray.rllib.policy.policy import Policy
from ray.rllib.utils.annotations import override
from ray.rllib.utils.deprecation import Deprecated
"""
from ray.rllib.utils.metrics import (
    LAST_TARGET_UPDATE_TS,
    NUM_AGENT_STEPS_SAMPLED,
    NUM_ENV_STEPS_SAMPLED,
    NUM_TARGET_UPDATES,
    SYNCH_WORKER_WEIGHTS_TIMER,
    SAMPLE_TIMER,
)
"""
from ray.rllib.utils.replay_buffers.utils import sample_min_n_steps_from_buffer
from ray.rllib.utils.typing import ResultDict
from ray.rllib.utils.deprecation import DEPRECATED_VALUE
from ray.rllib.utils.deprecation import deprecation_warning
#@ti.data_oriented
class QMixConfig(SimpleQConfig):
    """Defines a configuration class from which QMix can be built.
    Example:
        >>> from ray.rllib.examples.env.two_step_game import TwoStepGame
        >>> from ray.rllib.algorithms.qmix import QMixConfig
        >>> config = QMixConfig()  # doctest: +SKIP
        >>> config = config.training(gamma=0.9, lr=0.01, kl_coeff=0.3)  # doctest: +SKIP
        >>> config = config.resources(num_gpus=0)  # doctest: +SKIP
        >>> config = config.rollouts(num_rollout_workers=4)  # doctest: +SKIP
        >>> print(config.to_dict())  # doctest: +SKIP
        >>> # Build an Algorithm object from the config and run 1 training iteration.
        >>> algo = config.build(env=TwoStepGame)  # doctest: +SKIP
        >>> algo.train()  # doctest: +SKIP
    Example:
        >>> from ray.rllib.examples.env.two_step_game import TwoStepGame
        >>> from ray.rllib.algorithms.qmix import QMixConfig
        >>> from ray import air
        >>> from ray import tune
        >>> config = QMixConfig()
        >>> # Print out some default values.
        >>> print(config.optim_alpha)  # doctest: +SKIP
        >>> # Update the config object.
        >>> config.training(  # doctest: +SKIP
        ...     lr=tune.grid_search([0.001, 0.0001]), optim_alpha=0.97
        ... )
        >>> # Set the config object's env.
        >>> config.environment(env=TwoStepGame)  # doctest: +SKIP
        >>> # Use to_dict() to get the old-style python config dict
        >>> # when running with tune.
        >>> tune.Tuner(  # doctest: +SKIP
        ...     "QMix",
        ...     run_config=air.RunConfig(stop={"episode_reward_mean": 200}),
        ...     param_space=config.to_dict(),
        ... ).fit()
    """

    def __init__(self):
        """Initializes a PPOConfig instance."""
        super().__init__(algo_class=QMix)

        # fmt: off
        # __sphinx_doc_begin__
        # QMix specific settings:
        self.mixer = "qmix"
        self.mixing_embed_dim = 32
        self.double_q = True
        self.optim_alpha = 0.99
        self.optim_eps = 0.00001
        self.grad_clip = 10
        #self.render_mode = 'rgb_array'
        # QMix-torch overrides the TorchPolicy's learn_on_batch w/o specifying a
        # alternative `learn_on_loaded_batch` alternative for the GPU.
        # TODO: This hack will be resolved once we move all algorithms to the new
        #  RLModule/Learner APIs.
        self.simple_optimizer = True

        # Override some of AlgorithmConfig's default values with QMix-specific values.
        # .training()
        self.lr = 0.0005
        self.train_batch_size = 32
        self.target_network_update_freq = 500
        self.num_steps_sampled_before_learning_starts = 1000
        self.replay_buffer_config = {
            "type": "MultiAgentReplayBuffer",
            # Specify prioritized replay by supplying a buffer type that supports
            # prioritization, for example: MultiAgentPrioritizedReplayBuffer.
            "prioritized_replay": 'MultiAgentPrioritizedReplayBuffer',
            # Size of the replay buffer in batches (not timesteps!).
            "capacity": 1000,
            # Choosing `fragments` here makes it so that the buffer stores entire
            # batches, instead of sequences, episodes or timesteps.
            "storage_unit": "fragments",
            # Whether to compute priorities on workers.
            "worker_side_prioritization": False,
            #"no_local_replay_buffer": True,
        }
        """
        self.model = {
            "lstm_cell_size": 64,
            "max_seq_len": 999999,
        }
        """
        # .framework()
        self.framework_str = "torch"

        # .rollouts()
        self.rollout_fragment_length = 4
        self.batch_mode = "complete_episodes"

        # .reporting()
        self.min_time_s_per_iteration = 1
        self.min_sample_timesteps_per_iteration = 1000

        # .exploration()
        """
        self.exploration_config = {
           
            # The Exploration class to use.
            "type": "EpsilonGreedy", #replace this with SoftQ
            # Config for the Exploration class' constructor:
            "initial_epsilon": 1.0,
            "final_epsilon": 0.01,
            # Timesteps over which to anneal epsilon.
            "epsilon_timesteps": 40000,
            
            "type": "EFE",
            "temperature": 1.0
            # For soft_q, use:
            # "exploration_config" = {
            #   "type": "SoftQ"
            #   "temperature": [float, e.g. 1.0]
            # }
        }
        """

        # .evaluation()
        # Evaluate with epsilon=0 every `evaluation_interval` training iterations.
        # The evaluation stats will be reported under the "evaluation" metric key.
        self.evaluation(
            evaluation_config=AlgorithmConfig.overrides(explore=False)#False
        )
        # __sphinx_doc_end__
        # fmt: on

        self.worker_side_prioritization = DEPRECATED_VALUE

    @override(SimpleQConfig)
    def training(
        self,
        *,
        mixer: Optional[str] = NotProvided,
        mixing_embed_dim: Optional[int] = NotProvided,
        double_q: Optional[bool] = NotProvided,
        target_network_update_freq: Optional[int] = NotProvided,
        replay_buffer_config: Optional[dict] = NotProvided,
        optim_alpha: Optional[float] = NotProvided,
        optim_eps: Optional[float] = NotProvided,
        grad_clip: Optional[float] = NotProvided,
        # Deprecated args.
        grad_norm_clipping=DEPRECATED_VALUE,
        **kwargs,
    ) -> "QMixConfig":
        """Sets the training related configuration.
        Args:
            mixer: Mixing network. Either "qmix", "vdn", or None.
            mixing_embed_dim: Size of the mixing network embedding.
            double_q: Whether to use Double_Q learning.
            target_network_update_freq: Update the target network every
                `target_network_update_freq` sample steps.
            replay_buffer_config:
            optim_alpha: RMSProp alpha.
            optim_eps: RMSProp epsilon.
            grad_clip: If not None, clip gradients during optimization at
                this value.
            grad_norm_clipping: Depcrecated in favor of grad_clip
        Returns:
            This updated AlgorithmConfig object.
        """
        # Pass kwargs onto super's `training()` method.
        super().training(**kwargs)

        if grad_norm_clipping != DEPRECATED_VALUE:
            deprecation_warning(
                old="grad_norm_clipping",
                new="grad_clip",
                help="Parameter `grad_norm_clipping` has been "
                "deprecated in favor of grad_clip in QMix. "
                "This is now the same parameter as in other "
                "algorithms. `grad_clip` will be overwritten by "
                "`grad_norm_clipping={}`".format(grad_norm_clipping),
                error=True,
            )
            grad_clip = grad_norm_clipping

        if mixer is not NotProvided:
            self.mixer = mixer
        if mixing_embed_dim is not NotProvided:
            self.mixing_embed_dim = mixing_embed_dim
        if double_q is not NotProvided:
            self.double_q = double_q
        if target_network_update_freq is not NotProvided:
            self.target_network_update_freq = target_network_update_freq
        if replay_buffer_config is not NotProvided:
            self.replay_buffer_config = replay_buffer_config
        if optim_alpha is not NotProvided:
            self.optim_alpha = optim_alpha
        if optim_eps is not NotProvided:
            self.optim_eps = optim_eps
        if grad_clip is not NotProvided:
            self.grad_clip = grad_clip

        return self

    @override(SimpleQConfig)
    def validate(self) -> None:
        # Call super's validation method.
        super().validate()

        if self.framework_str != "torch":
            raise ValueError(
                "Only `config.framework('torch')` supported so far for QMix!"
            )

class QMix(SimpleQ):
    @classmethod
    @override(SimpleQ)
    def get_default_config(cls) -> AlgorithmConfig:
        return QMixConfig()

    @classmethod
    @override(SimpleQ)
    def get_default_policy_class(
        cls, config: AlgorithmConfig
    ) -> Optional[Type[Policy]]:
        return QMixTorchPolicy

    @override(SimpleQ)
    def training_step(self) -> ResultDict:
        """QMIX training iteration function.
        - Sample n MultiAgentBatches from n workers synchronously.
        - Store new samples in the replay buffer.
        - Sample one training MultiAgentBatch from the replay buffer.
        - Learn on the training batch.
        - Update the target network every `target_network_update_freq` sample steps.
        - Return all collected training metrics for the iteration.
        Returns:
            The results dict from executing the training iteration.
        """
        # Sample n batches from n workers.
        with self._timers[SAMPLE_TIMER]:
            new_sample_batches = synchronous_parallel_sample(
                worker_set=self.workers, concat=False
            )

        for batch in new_sample_batches:
            # Update counters.
            self._counters[NUM_ENV_STEPS_SAMPLED] += batch.env_steps()
            self._counters[NUM_AGENT_STEPS_SAMPLED] += batch.agent_steps()
            # Store new samples in the replay buffer.
            self.local_replay_buffer.add(batch)

        # Update target network every `target_network_update_freq` sample steps.
        cur_ts = self._counters[
            NUM_AGENT_STEPS_SAMPLED
            if self.config.count_steps_by == "agent_steps"
            else NUM_ENV_STEPS_SAMPLED
        ]

        train_results = {}

        if cur_ts > self.config.num_steps_sampled_before_learning_starts:
            # Sample n batches from replay buffer until the total number of timesteps
            # reaches `train_batch_size`.
            train_batch = sample_min_n_steps_from_buffer(
                replay_buffer=self.local_replay_buffer,
                min_steps=self.config.train_batch_size,
                count_by_agent_steps=self.config.count_steps_by == "agent_steps",
            )

            # Learn on the training batch.
            # Use simple optimizer (only for multi-agent or tf-eager; all other
            # cases should use the multi-GPU optimizer, even if only using 1 GPU)
            if self.config.get("simple_optimizer") is True:
                train_results = train_one_step(self, train_batch)
            else:
                train_results = multi_gpu_train_one_step(self, train_batch)

            # Update target network every `target_network_update_freq` sample steps.
            last_update = self._counters[LAST_TARGET_UPDATE_TS]
            if cur_ts - last_update >= self.config.target_network_update_freq:
                to_update = self.workers.local_worker().get_policies_to_train()
                self.workers.local_worker().foreach_policy_to_train(
                    lambda p, pid: pid in to_update and p.update_target()
                )
                self._counters[NUM_TARGET_UPDATES] += 1
                self._counters[LAST_TARGET_UPDATE_TS] = cur_ts

            update_priorities_in_replay_buffer(
                self.local_replay_buffer, self.config, train_batch, train_results
            )

            # Update weights and global_vars - after learning on the local worker -
            # on all remote workers.
            global_vars = {
                "timestep": self._counters[NUM_ENV_STEPS_SAMPLED],
            }
            # Update remote workers' weights and global vars after learning on local
            # worker.
            with self._timers[SYNCH_WORKER_WEIGHTS_TIMER]:
                self.workers.sync_weights(global_vars=global_vars)

        # Return all collected metrics for the iteration.
        return train_results

In [19]:
from gym.spaces import Dict, Discrete, Tuple, MultiDiscrete
#b = np.ones(3000).tolist()
#obs_space= Tuple({MultiDiscrete([])})
"""
observation_space = Tuple(
    [
        Dict(
            {
                "obs": MultiDiscrete([3,3]),#sensing whether it got inhibitory, excitatory or none
                ENV_STATE: MultiDiscrete([3,3]),
            }
        ),
        Dict(
            {
                "obs": MultiDiscrete([3,3]),
                ENV_STATE: MultiDiscrete([3,3]),
            }
        ),
    ]
)

action_space = Tuple(
    [
        Dict(
            {
                "obs": MultiDiscrete([1,3])#MultiDiscrete([1,3]),
                #ENV_STATE: MultiDiscrete([3,1]),
            }
        ),
        Dict(
            {
                "obs": MultiDiscrete([3,1]),#3,1
                #ENV_STATE: MultiDiscrete([3,1]),
            }
        ),
    ]
)
obs_space = Tuple([#
    Dict({
        "obs": MultiDiscrete([2, 2, 2, 3]),
        ENV_STATE: MultiDiscrete([2, 2, 2]),
                    
    }
    )
]
)
"""



'\nobservation_space = Tuple(\n    [\n        Dict(\n            {\n                "obs": MultiDiscrete([3,3]),#sensing whether it got inhibitory, excitatory or none\n                ENV_STATE: MultiDiscrete([3,3]),\n            }\n        ),\n        Dict(\n            {\n                "obs": MultiDiscrete([3,3]),\n                ENV_STATE: MultiDiscrete([3,3]),\n            }\n        ),\n    ]\n)\n\naction_space = Tuple(\n    [\n        Dict(\n            {\n                "obs": MultiDiscrete([1,3])#MultiDiscrete([1,3]),\n                #ENV_STATE: MultiDiscrete([3,1]),\n            }\n        ),\n        Dict(\n            {\n                "obs": MultiDiscrete([3,1]),#3,1\n                #ENV_STATE: MultiDiscrete([3,1]),\n            }\n        ),\n    ]\n)\nobs_space = Tuple([#\n    Dict({\n        "obs": MultiDiscrete([2, 2, 2, 3]),\n        ENV_STATE: MultiDiscrete([2, 2, 2]),\n                    \n    }\n    )\n]\n)\n'

In [66]:
from ray.rllib.utils.replay_buffers.multi_agent_replay_buffer import MultiAgentReplayBuffer
from ray.rllib.utils.replay_buffers.multi_agent_prioritized_replay_buffer import MultiAgentPrioritizedReplayBuffer
from ray.rllib.utils.replay_buffers import ReplayBuffer, StorageUnit 
from ray.rllib.utils.replay_buffers.utils import sample_min_n_steps_from_buffer

In [20]:
from gymnasium.envs.registration import register


In [21]:
from ray.tune.registry import register_env
#env_config={}
def env_creator(env_config):
    return Pegasus()

register_env("Pegasus", env_creator)

In [31]:
from ray.tune.registry import register_env
#env_config={"n_agents":3000}
def env_creator(env_config):
    return HierarchicalGroupedPegasus()

#register_env("HierarchicalGroupedPegasus", env_creator)
register_env("HierarchicalGroupedPegasus", lambda config: HierarchicalGroupedPegasus(config))

In [110]:
from ray.tune.registry import register_env
#env_config={"n_agents":3000}
def env_creator(env_config):
    return TwoStepGameWithGroupedAgents()

#register_env("HierarchicalGroupedPegasus", env_creator)
register_env("TwoStepGameWithGroupedAgents", lambda config: TwoStepGameWithGroupedAgents(config))

In [23]:
print(HierarchicalGroupedPegasus())

TypeError: __init__() missing 1 required positional argument: 'env_config'

In [19]:
import argparse
from gymnasium.spaces import Dict, Discrete, Tuple, MultiDiscrete
import logging
import os
from ray.rllib.examples.env.two_step_game import TwoStepGame
import ray
from ray import air, tune
from ray.tune import register_env
from ray.rllib.env.multi_agent_env import ENV_STATE
#from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import get_trainable_cls
from gymnasium.spaces import Dict, Discrete, MultiDiscrete, Tuple,Box
logger = logging.getLogger(__name__)
from ray.tune import register_env

parser = argparse.ArgumentParser()
parser.add_argument(
    "--run", type=str, default="QMIX", help="The RLlib-registered algorithm to use."
)
parser.add_argument(
    "--framework",
    choices=["tf", "tf2", "torch"],
    default="torch",
    help="The DL framework specifier.",
)

parser.add_argument(
    "--mixer",
    type=str,
    default="qmix",
    choices=["qmix", "vdn", "none"],
    help="The mixer model to use.",
)
parser.add_argument("--num-cpus", type=int, default=14)
#parser.add_argument("--num-gpus", type=int, default=1)
parser.add_argument("--num-gpus", type=float, default=1)#0.25)

parser.add_argument("--num-workers", type=int, default=6)
parser.add_argument("--num-gpus-per-worker", type=float, default=0)
parser.add_argument(
    "--as-test",
    action="store_true",
    help="Whether this script should be run as a test: --stop-reward must "
    "be achieved within --stop-timesteps AND --stop-iters.",
)
parser.add_argument(
    "--stop-iters", type=int, default=200, help="Number of iterations to train."
)
parser.add_argument(
    "--stop-timesteps", type=int, default=70000, help="Number of timesteps to train."
)
parser.add_argument(
    "--stop-reward", type=float, default=6.0, help="Reward at which we stop training."
)#8 
#parser.add_argument("--num-workers", type=int, default=6)
parser.add_argument("--num_envs_per_worker", type=int,default=1)
parser.add_argument(
    "--local-mode",
    action="store_true",
    help="Init Ray in local mode for easier debugging.",
)

if __name__ == "__main__":
    args = parser.parse_args()

    ray.init(num_cpus=args.num_cpus or None, num_gpus=1,local_mode=args.local_mode)
    
    #register_env(Pegasus,HierarchicalGroupedPegasus)

    grouping = {
        "group_1": [0,1],
    }
    obs_space = Tuple(
        [
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
        ]
    )
    act_space = Tuple(
        [
            TwoStepGame.action_space,
            TwoStepGame.action_space,
        ]
    )

    #register_env("grouped_twostep",lambda config: TwoStepGame(config).with_agent_groups(grouping, obs_space=obs_space, act_space=act_space),)

    """
    register_env(
        "'HierarchicalGroupedPegasus'",
        lambda config: Pegasus(config).with_agent_groups(
            grouping, obs_space=obs_space, act_space=act_space
        ),
    )

    
    
    """
    def policy_mapping_fn(agent_id, episode, worker, **kwargs):
        #if agent_id.startswith("low_level_"):
        if agent_id.startswith("group_1"):
            return "low_level_policy"
        else:
            return "high_level_policy"

    config = (
        get_trainable_cls(args.run)
        .get_default_config()
        .environment('Pegasus')
        .framework(args.framework)
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )
    if args.run == "QMIX":
        (
            config.framework("torch")
            .training(mixer=args.mixer, train_batch_size=32)
            .rollouts(num_rollout_workers=args.num_workers, rollout_fragment_length=4)
            .exploration(#rollout_fragment_length=4
                exploration_config={
                    "final_epsilon": 0.0,
                }
            )
            .environment(
                env='HierarchicalGroupedPegasus',#HierarchicalGroupedPegasus(),#"Pegasus",#HierarchicalGroupedPegasus
                env_config={
                    "separate_state_space": True,
                    "one_hot_state_encoding": True,
                },
            )
        )

    stop = {
        "episode_reward_mean": args.stop_reward,
        "timesteps_total": args.stop_timesteps,
        "training_iteration": args.stop_iters,
    }
    from ray.tune.schedulers import PopulationBasedTraining

    pbt_scheduler = PopulationBasedTraining(
        time_attr='training_iteration',
        metric='episode_reward_mean',#'loss',
        mode='min',
        perturbation_interval=1,
        hyperparam_mutations={
            #"lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
            "alpha": tune.uniform(0.0, 1.0),
        }
    )

    results = tune.Tuner(
        args.run,
        run_config=air.RunConfig(stop=stop, verbose=2),
        tune_config=tune.TuneConfig(
            num_samples=2,#4,
            scheduler=pbt_scheduler,
            #reuse_actors=True,
        ),
        param_space=config,
    ).fit()

    if args.as_test:
        check_learning_achieved(results, args.stop_reward)

    ray.shutdown()

KeyboardInterrupt: 

In [26]:
ray.shutdown()

In [42]:
import argparse
from gymnasium.spaces import Dict, Discrete, Tuple, MultiDiscrete
import logging
import os
from ray.rllib.examples.env.two_step_game import TwoStepGame
import ray
from ray import air, tune
from ray.tune import register_env
from ray.rllib.env.multi_agent_env import ENV_STATE
#from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import get_trainable_cls
from gymnasium.spaces import Dict, Discrete, MultiDiscrete, Tuple,Box

In [46]:
import argparse
from gymnasium.spaces import Dict, Discrete, Tuple, MultiDiscrete
import logging
import os
from ray.rllib.examples.env.two_step_game import TwoStepGame
import ray
from ray import air, tune
from ray.tune import register_env
from ray.rllib.env.multi_agent_env import ENV_STATE
#from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import get_trainable_cls
from gymnasium.spaces import Dict, Discrete, MultiDiscrete, Tuple,Box
logger = logging.getLogger(__name__)

parser = argparse.ArgumentParser()
parser.add_argument(
    "--run", type=str, default="QMIX", help="The RLlib-registered algorithm to use."
)
parser.add_argument(
    "--framework",
    choices=["tf", "tf2", "torch"],
    default="torch",
    help="The DL framework specifier.",
)

parser.add_argument(
    "--mixer",
    type=str,
    default="qmix",
    choices=["qmix", "vdn", "none"],
    help="The mixer model to use.",
)
parser.add_argument("--num-cpus", type=int, default=13)
#parser.add_argument("--num-gpus", type=int, default=1)
parser.add_argument("--num-gpus", type=float, default=1)#0.25)

parser.add_argument("--num-workers", type=int, default=13)
parser.add_argument("--num-gpus-per-worker", type=float, default=0)
parser.add_argument(
    "--as-test",
    action="store_true",
    help="Whether this script should be run as a test: --stop-reward must "
    "be achieved within --stop-timesteps AND --stop-iters.",
)
parser.add_argument(
    "--stop-iters", type=int, default=200, help="Number of iterations to train."
)
parser.add_argument(
    "--stop-timesteps", type=int, default=70000, help="Number of timesteps to train."
)
parser.add_argument(
    "--stop-reward", type=float, default=6.0, help="Reward at which we stop training."
)#8 
#parser.add_argument("--num-workers", type=int, default=6)
parser.add_argument("--num_envs_per_worker", type=int,default=1)
parser.add_argument(
    "--local-mode",
    action="store_true",
    help="Init Ray in local mode for easier debugging.",
)

if __name__ == "__main__":
    args = parser.parse_args()

    ray.init(num_cpus=args.num_cpus or None, num_gpus=1,local_mode=args.local_mode)

    grouping = {
        "group_1": [0,1],
    }
    obs_space = Tuple(
        [
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
        ]
    )
    act_space = Tuple(
        [
            TwoStepGame.action_space,
            TwoStepGame.action_space,
        ]
    )

    register_env(
        "grouped_twostep",
        lambda config: TwoStepGame(config).with_agent_groups(
            grouping, obs_space=obs_space, act_space=act_space
        ),
    )

    def policy_mapping_fn(agent_id, episode, worker, **kwargs):
        #if agent_id.startswith("low_level_"):
        if agent_id.startswith("group_1"):
            return "low_level_policy"
        else:
            return "high_level_policy"

    config = (
        get_trainable_cls(args.run)
        .get_default_config()
        .environment(TwoStepGame)
        .framework(args.framework)
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )
    if args.run == "QMIX":
        (
            config.framework("torch")
            .training(mixer=args.mixer, train_batch_size=32)# increasing this from 32 should help us use our gpu
            .rollouts(num_rollout_workers=12, rollout_fragment_length=4)#args.num_workers
            .resources(num_gpus=1)
            .exploration(#rollout_fragment_length=4
                exploration_config={
                    "final_epsilon": 0.0,
                }
            )
            .environment(
                env="grouped_twostep",
                env_config={
                    "separate_state_space": True,
                    "one_hot_state_encoding": True,
                },
            )
        )

    stop = {
        "episode_reward_mean": args.stop_reward,
        "timesteps_total": args.stop_timesteps,
        "training_iteration": args.stop_iters,
    }
    from ray.tune.schedulers import PopulationBasedTraining

    pbt_scheduler = PopulationBasedTraining(
        time_attr='training_iteration',
        metric='episode_reward_mean',#'loss',
        mode='min',
        perturbation_interval=1,
        hyperparam_mutations={
            #"lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
            "alpha": tune.uniform(0.0, 1.0),
        }
    )

    results = tune.Tuner(
        args.run,
        run_config=air.RunConfig(stop=stop, verbose=2),
        tune_config=tune.TuneConfig(
            num_samples=1,#4,
            scheduler=pbt_scheduler,
            #reuse_actors=True,
        ),
        param_space=config,
    ).fit()

    if args.as_test:
        check_learning_achieved(results, args.stop_reward)

    ray.shutdown()
    """
    if args.run == "QMIX":

        (
            config.framework("torch")
            .environment(env_config={"actions_are_logits": True})#line below was causing problems
            .training(mixer=args.mixer, train_batch_size=32)#.training(num_steps_sampled_before_learning_starts=100)
            .multi_agent(
                policies={
                    "high_level_policy": (
                        None,
                        observation_space,
                        action_space,
                        config.overrides(gamma=0.9),
                    ),
                    "low_level_policy": (
                        None,
                        #Tuple([observation_space,Box(-inf, inf, (1,), float64)]),#,Box()
                        Tuple([observation_space,obs_space]),
                        Tuple([action_space,obs_space]),#action_space,
                        config.overrides(gamma=0.0),
                    ),
                },
                policy_mapping_fn=policy_mapping_fn#policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "pol2"
            )
            .rollouts(num_rollout_workers=args.num_workers, num_envs_per_worker=args.num_envs_per_worker)#.rollouts(num_rollout_workers=args.num_workers, num_envs_per_worker=args.num_envs_per_worker,)
            .exploration(
                exploration_config={
                    "final_epsilon": 0.0,
                }
            )
            
            .environment(
                env="grouped_twostep",
                env_config={
                    "separate_state_space": True,
                    "one_hot_state_encoding": True,
                },
            )
        )

    stop = {
        "episode_reward_mean": args.stop_reward,
        "timesteps_total": args.stop_timesteps,
        "training_iteration": args.stop_iters,
    }

    results = tune.Tuner(
        args.run,
        run_config=air.RunConfig(stop=stop, verbose=2),
        tune_config=tune.TuneConfig(
            num_samples=4,
            scheduler=pbt_scheduler,
            #reuse_actors=True,
        ),
        
        param_space=config,
    ).fit()

    if args.as_test:
        check_learning_achieved(results, args.stop_reward)

    ray.shutdown()
    """

2023-08-04 18:18:16,279	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
C:\Users\subar\anaconda3\envs\d\lib\site-packages\ray\tune\tune.py:562: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
  warnings.warn(


2023-08-04 18:18:18,027	WARNING trial_runner.py:1677 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler to access this TrialRunner API, please reach out to Ray team on GitHub. A more strict API access pattern would be enforced starting 1.12s.0
(pid=28916) Windows fatal exception: code 0xc0000139
(pid=28916) 
(pid=28916) Windows fatal exception: code 0xc0000139
(pid=28916) 
(pid=28916) Windows fatal exception: code 0xc0000139
(pid=28916) 
(pid=28916) Windows fatal exception: code 0xc0000139
(pid=28916) 
(pid=28916) Windows fatal exception: code 0xc0000139
(pid=28916) 
(pid=28916) Windows fatal exception: code 0xc0000139
(pid=28916) 
(pid=28916) Windows fatal exception: code 0xc0000139
(pid=28916) 
(pid=28916) Windows fatal exception: code 0xc0000139
(pid=28916) 
(pid=28916) Windows fatal exception: code 0xc0000139
(pid=28916) 
(pid=28916) Stack (most recent call first):
(pid=28

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,timers
QMIX_grouped_twostep_f575c_00000,20160,"{'ObsPreprocessorConnector_ms': 0.16034675969017875, 'StateBufferConnector_ms': 0.030730215329972523, 'ViewRequirementAgentConnector_ms': 0.4713467662296598}","{'num_env_steps_sampled': 20160, 'num_env_steps_trained': 12800, 'num_agent_steps_sampled': 20160, 'num_agent_steps_trained': 12800, 'last_target_update_ts': 20016, 'num_target_updates': 37}",{},2,{},8,6.3869,0,504,"{'learner': {'default_policy': {'learner_stats': {'loss': 0.00015566678484901786, 'td_error_abs': 0.011360058560967445, 'q_taken_mean': 0.1945074498653412, 'target_mean': 0.1831473857164383, 'grad_gnorm': 0.16391624510288239}}}, 'num_env_steps_sampled': 20160, 'num_env_steps_trained': 12800, 'num_agent_steps_sampled': 20160, 'num_agent_steps_trained': 12800, 'last_target_update_ts': 20016, 'num_target_updates': 37}",20160,12800,20160,1008,12800,672,0,12,0,0,672,"{'cpu_util_percent': 15.3, 'ram_util_percent': 69.80000000000001, 'gpu_util_percent0': 0.16999999999999998, 'vram_util_percent0': 0.302978515625}",{},{},{},"{'mean_raw_obs_processing_ms': 1.8747017871948217, 'mean_inference_ms': 1.297153935838072, 'mean_action_processing_ms': 0.28678396025655195, 'mean_env_wait_ms': 0.10116950590506965, 'mean_env_render_ms': 0.0}","{'training_iteration_time_ms': 72.62, 'learn_time_ms': 26.598, 'learn_throughput': 1203.116, 'synch_weights_time_ms': 17.02}"


2023-08-04 18:19:26,427	INFO tune.py:798 -- Total run time: 68.46 seconds (67.81 seconds for the tuning loop).


In [47]:
config = {}
a = TwoStepGame(config).with_agent_groups(
            grouping, obs_space=obs_space, act_space=act_space)

In [60]:
print(a.reset())#initial observations
obs,info = a.reset()

print(obs)
print(info)

print(algo.get_policy().compute_actions())#compute_single_action(obs))

({'group_1': [0, 3]}, {})
{'group_1': [0, 3]}
{}


AttributeError: 'NoneType' object has no attribute 'compute_actions'

In [ ]:
#let us do a small scale visualization demonstration of qamix two step first 



In [45]:
ray.shutdown()

In [31]:
import argparse
from gymnasium.spaces import Dict, Discrete, Tuple, MultiDiscrete
import logging
import os

import ray
from ray import air, tune
from ray.tune import register_env
from ray.rllib.env.multi_agent_env import ENV_STATE
from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import get_trainable_cls

logger = logging.getLogger(__name__)

parser = argparse.ArgumentParser()
parser.add_argument(
    "--run", type=str, default="PG", help="The RLlib-registered algorithm to use."
)
parser.add_argument(
    "--framework",
    choices=["tf", "tf2", "torch"],
    default="torch",
    help="The DL framework specifier.",
)
parser.add_argument("--num-cpus", type=int, default=14)
parser.add_argument("--num-gpus", type=float, default=0.5)

parser.add_argument(
    "--mixer",
    type=str,
    default="qmix",
    choices=["qmix", "vdn", "none"],
    help="The mixer model to use.",
)
parser.add_argument("--num-envs-per-worker", type=int, default=1)
parser.add_argument(
    "--as-test",
    action="store_true",
    help="Whether this script should be run as a test: --stop-reward must "
    "be achieved within --stop-timesteps AND --stop-iters.",
)
parser.add_argument(
    "--stop-iters", type=int, default=200, help="Number of iterations to train."
)
parser.add_argument(
    "--stop-timesteps", type=int, default=70000, help="Number of timesteps to train."
)
parser.add_argument(
    "--stop-reward", type=float, default=8.0, help="Reward at which we stop training."
)
parser.add_argument(
    "--local-mode",
    action="store_true",
    help="Init Ray in local mode for easier debugging.",
)

if __name__ == "__main__":
    args = parser.parse_args()

    ray.init(num_cpus=14, local_mode=args.local_mode)

    grouping = {
        "group_1": [0,1],
    }
    def policy_mapping_fn(agent_id, episode, worker, **kwargs):
        #if agent_id.startswith("low_level_"):
        if agent_id.startswith(0):
            return "low_level_policy"
        else:
            return "high_level_policy"
    obs_space = Tuple(
        [
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
        ]
    )
    act_space = Tuple(
        [
            TwoStepGame.action_space,
            TwoStepGame.action_space,
        ]
    )
    register_env(
        "grouped_twostep",
        lambda config: TwoStepGame(config).with_agent_groups(
            grouping, obs_space=obs_space, act_space=act_space
        ),
    )

    config = (
        get_trainable_cls(args.run)
        .get_default_config()
        .environment(TwoStepGame)
        .framework(args.framework)
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )

    if args.run == "MADDPG":
        obs_space = Discrete(6)
        act_space = TwoStepGame.action_space
        (
            config.framework("tf")
            .environment(env_config={"actions_are_logits": True})
            .training(num_steps_sampled_before_learning_starts=100)
            .multi_agent(
                policies={
                    "pol1": PolicySpec(
                        observation_space=obs_space,
                        action_space=act_space,
                        config=config.overrides(agent_id=0),
                    ),
                    "pol2": PolicySpec(
                        observation_space=obs_space,
                        action_space=act_space,
                        config=config.overrides(agent_id=1),
                    ),
                },
                policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "pol2"
                if agent_id
                else "pol1",
            )
        )
    elif args.run == "QMIX":
        (
            config.framework("torch")
            .training(mixer=args.mixer, train_batch_size=32)
            
            .rollouts(num_rollout_workers=0, rollout_fragment_length=4)
            .exploration(
                exploration_config={
                    "final_epsilon": 0.0,
                }
            )
            .environment(
                env="grouped_twostep",
                env_config={
                    "separate_state_space": True,
                    "one_hot_state_encoding": True,
                },
            )
        )

    stop = {
        "episode_reward_mean": args.stop_reward,
        "timesteps_total": args.stop_timesteps,
        "training_iteration": args.stop_iters,
    }

    results = tune.Tuner(
        args.run,
        run_config=air.RunConfig(stop=stop, verbose=2),
        param_space=config,
    ).fit()

    if args.as_test:
        check_learning_achieved(results, args.stop_reward)

    ray.shutdown()

2023-07-17 19:05:13,473	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


(pid=44920) Windows fatal exception: code 0xc0000139
(pid=44920) 
(pid=44920) Windows fatal exception: code 0xc0000139
(pid=44920) 
(pid=44920) Windows fatal exception: code 0xc0000139
(pid=44920) 
(pid=44920) Windows fatal exception: code 0xc0000139
(pid=44920) 
(pid=44920) Windows fatal exception: code 0xc0000139
(pid=44920) 
(pid=44920) Windows fatal exception: code 0xc0000139
(pid=44920) 
(pid=44920) Windows fatal exception: code 0xc0000139
(pid=44920) 
(pid=44920) Windows fatal exception: code 0xc0000139
(pid=44920) 
(pid=44920) C:\Users\subar\anaconda3\envs\d\lib\site-packages\gym\envs\registration.py:307: DeprecationWarning: The package name gym_minigrid has been deprecated in favor of minigrid. Please uninstall gym_minigrid and install minigrid with `pip install minigrid`. Future releases will be maintained under the new package name minigrid.
(pid=44920)   fn()
(pid=44920) Windows fatal exception: code 0xc0000139
(pid=44920) 
(pid=44920) Windows fatal exception: code 0xc000013

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PG_TwoStepGame_8903b_00000,80000,"{'StateBufferConnector_ms': 0.009002923965454102, 'ViewRequirementAgentConnector_ms': 0.12249398231506348}","{'num_env_steps_sampled': 40000, 'num_env_steps_trained': 40000, 'num_agent_steps_sampled': 80000, 'num_agent_steps_trained': 80000}",{},2,{},8,7.03,7,100,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'policy_loss': 1.0265274345874786, 'cur_lr': 0.0004}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 200.0, 'num_grad_updates_lifetime': 399.5, 'diff_num_grad_updates_vs_sampler_policy': 0.5}}, 'num_env_steps_sampled': 40000, 'num_env_steps_trained': 40000, 'num_agent_steps_sampled': 80000, 'num_agent_steps_trained': 80000}",80000,80000,40000,200,40000,200,0,0,0,0,200,"{'cpu_util_percent': 6.6, 'ram_util_percent': 49.9, 'gpu_util_percent0': 0.0, 'vram_util_percent0': 0.2060546875}",{},{},{},"{'mean_raw_obs_processing_ms': 0.5982967887007353, 'mean_inference_ms': 0.8197147659485234, 'mean_action_processing_ms': 0.10617833528389693, 'mean_env_wait_ms': 0.0351240258166672, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 8.0, 'episode_reward_min': 7.0, 'episode_reward_mean': 7.03, 'episode_len_mean': 2.0, 'episode_media': {}, 'episodes_this_iter': 100, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 8.0, 7.0, 7.0, 7.0, 7.0, 8.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 8.0, 7.0, 7.0, 7.0, 7.0], 'episode_lengths': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.5982967887007353, 'mean_inference_ms': 0.8197147659485234, 'mean_action_processing_ms': 0.10617833528389693, 'mean_env_wait_ms': 0.0351240258166672, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'StateBufferConnector_ms': 0.009002923965454102, 'ViewRequirementAgentConnector_ms': 0.12249398231506348}}","{'training_iteration_time_ms': 323.555, 'load_time_ms': 0.3, 'load_throughput': 666980.043, 'learn_time_ms': 6.603, 'learn_throughput': 30288.485, 'synch_weights_time_ms': 0.1}"


2023-07-17 19:06:33,402	INFO tune.py:798 -- Total run time: 78.55 seconds (77.92 seconds for the tuning loop).


In [28]:
ray.shutdown()

In [9]:

import argparse
from gymnasium.spaces import Dict, Discrete, Tuple, MultiDiscrete
import logging
import os

import ray
from ray import air, tune
from ray.tune import register_env
from ray.rllib.env.multi_agent_env import ENV_STATE
from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import get_trainable_cls

logger = logging.getLogger(__name__)

parser = argparse.ArgumentParser()
parser.add_argument(
    "--run", type=str, default="QMIX", help="The RLlib-registered algorithm to use."
)
parser.add_argument(
    "--framework",
    choices=["tf", "tf2", "torch"],
    default="torch",
    help="The DL framework specifier.",
)
parser.add_argument("--num-cpus", type=int, default=0)
parser.add_argument(
    "--mixer",
    type=str,
    default="qmix",
    choices=["qmix", "vdn", "none"],
    help="The mixer model to use.",
)
parser.add_argument(
    "--as-test",
    action="store_true",
    help="Whether this script should be run as a test: --stop-reward must "
    "be achieved within --stop-timesteps AND --stop-iters.",
)
parser.add_argument(
    "--stop-iters", type=int, default=200, help="Number of iterations to train."
)
parser.add_argument(
    "--stop-timesteps", type=int, default=70000, help="Number of timesteps to train."
)
parser.add_argument(
    "--stop-reward", type=float, default=8.0, help="Reward at which we stop training."
)
parser.add_argument(
    "--local-mode",
    action="store_true",
    help="Init Ray in local mode for easier debugging.",
)

if __name__ == "__main__":
    args = parser.parse_args()

    ray.init(num_cpus=args.num_cpus or None, num_gpus=1,local_mode=args.local_mode)

    grouping = {
        "group_1": [0, 1],
    }
    
    obs_space = Tuple(
        [
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
        ]
    )
    act_space = Tuple(
        [
            TwoStepGame.action_space,
            TwoStepGame.action_space,
        ]
    )
    register_env(
        "grouped_twostep",
        lambda config: TwoStepGame(config).with_agent_groups(
            grouping, obs_space=obs_space, act_space=act_space
        ),
    )

    config = (
        get_trainable_cls(args.run)
        .get_default_config()
        .environment("grouped_twostep")
        .framework(args.framework)
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )

    if args.run == "QMIX":
        observation_space = Tuple([Discrete(6)])
        act_space = Tuple([TwoStepGame.action_space]) 
        (
            config.framework("torch")
            .environment(env_config={"actions_are_logits": True})
            .training(num_steps_sampled_before_learning_starts=100)


    stop = {
        "episode_reward_mean": args.stop_reward,
        "timesteps_total": args.stop_timesteps,
        "training_iteration": args.stop_iters,
    }

    results = tune.Tuner(
        args.run,
        run_config=air.RunConfig(stop=stop, verbose=2),
        param_space=config,
    ).fit()

    if args.as_test:
        check_learning_achieved(results, args.stop_reward)

    ray.shutdown()


2023-07-04 15:46:11,396	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 


(pid=47488) Windows fatal exception: code 0xc0000139
(pid=47488) 
(pid=47488) Windows fatal exception: code 0xc0000139
(pid=47488) 
(pid=47488) Windows fatal exception: code 0xc0000139
(pid=47488) 
(pid=47488) Windows fatal exception: code 0xc0000139
(pid=47488) 
(pid=47488) Windows fatal exception: code 0xc0000139
(pid=47488) 
(pid=47488) Windows fatal exception: code 0xc0000139
(pid=47488) 
(pid=47488) Windows fatal exception: code 0xc0000139
(pid=47488) 
(pid=47488) Windows fatal exception: code 0xc0000139
(pid=47488) 
(pid=47488) C:\Users\subar\anaconda3\envs\d\lib\site-packages\gym\envs\registration.py:307: DeprecationWarning: The package name gym_minigrid has been deprecated in favor of minigrid. Please uninstall gym_minigrid and install minigrid with `pip install minigrid`. Future releases will be maintained under the new package name minigrid.
(pid=47488)   fn()
(pid=47488) Windows fatal exception: code 0xc0000139
(pid=47488) 
(pid=47488) Windows fatal exception: code 0xc000013

Trial name
QMIX_grouped_twostep_9484b_00000


2023-07-04 15:46:26,074	ERROR tune.py:794 -- Trials did not complete: [QMIX_grouped_twostep_9484b_00000]
2023-07-04 15:46:26,075	INFO tune.py:798 -- Total run time: 11.78 seconds (11.07 seconds for the tuning loop).


In [36]:
ray.shutdown()

In [23]:
import argparse
from gymnasium.spaces import Dict, Discrete, Tuple, MultiDiscrete
import logging
import os

import ray
from ray import air, tune
from ray.tune import register_env
from ray.rllib.env.multi_agent_env import ENV_STATE
from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import get_trainable_cls

logger = logging.getLogger(__name__)

parser = argparse.ArgumentParser()
parser.add_argument(
    "--run", type=str, default="QMIX", help="The RLlib-registered algorithm to use."
)
parser.add_argument(
    "--framework",
    choices=["tf", "tf2", "torch"],
    default="torch",
    help="The DL framework specifier.",
)
parser.add_argument("--num-cpus", type=int, default=0)
parser.add_argument(
    "--mixer",
    type=str,
    default="qmix",
    choices=["qmix", "vdn", "none"],
    help="The mixer model to use.",
)
parser.add_argument(
    "--as-test",
    action="store_true",
    help="Whether this script should be run as a test: --stop-reward must "
    "be achieved within --stop-timesteps AND --stop-iters.",
)
parser.add_argument(
    "--stop-iters", type=int, default=200, help="Number of iterations to train."
)
parser.add_argument(
    "--stop-timesteps", type=int, default=70000, help="Number of timesteps to train."
)
parser.add_argument(
    "--stop-reward", type=float, default=8.0, help="Reward at which we stop training."
)
parser.add_argument(
    "--local-mode",
    action="store_true",
    help="Init Ray in local mode for easier debugging.",
)

if __name__ == "__main__":
    args = parser.parse_args()

    ray.init(num_cpus=args.num_cpus or None, local_mode=args.local_mode)

    grouping = {
        "group_1": [0,1],
    }
    def policy_mapping_fn(agent_id, episode, worker, **kwargs):
        #if agent_id.startswith("low_level_"):
        if agent_id.startswith(0):
            return "low_level_policy"
        else:
            return "high_level_policy"
    obs_space = Tuple(
        [
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
        ]
    )
    act_space = Tuple(
        [
            TwoStepGame.action_space,
            TwoStepGame.action_space,
        ]
    )
    register_env(
        "grouped_twostep",
        lambda config: TwoStepGame(config).with_agent_groups(
            grouping, obs_space=obs_space, act_space=act_space
        ),
    )

    config = (
        get_trainable_cls(args.run)
        .get_default_config()
        .environment(TwoStepGame)
        .framework(args.framework)
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )

    if args.run == "MADDPG":
        obs_space = Discrete(6)
        act_space = TwoStepGame.action_space
        (
            config.framework("tf")
            .environment(env_config={"actions_are_logits": True})
            .training(num_steps_sampled_before_learning_starts=100)
            .multi_agent(
                policies={
                    "pol1": PolicySpec(
                        observation_space=obs_space,
                        action_space=act_space,
                        config=config.overrides(agent_id=0),
                    ),
                    "pol2": PolicySpec(
                        observation_space=obs_space,
                        action_space=act_space,
                        config=config.overrides(agent_id=1),
                    ),
                },
                policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "pol2"
                if agent_id
                else "pol1",
            )
        )
    elif args.run == "QMIX":
        (
            config.framework("torch")
            .training(mixer=args.mixer, train_batch_size=32)
            
            .rollouts(num_rollout_workers=0, rollout_fragment_length=4)
            .exploration(
                exploration_config={
                    "final_epsilon": 0.0,
                }
            )
            .environment(
                env="grouped_twostep",
                env_config={
                    "separate_state_space": True,
                    "one_hot_state_encoding": True,
                },
            )
        )

    stop = {
        "episode_reward_mean": args.stop_reward,
        "timesteps_total": args.stop_timesteps,
        "training_iteration": args.stop_iters,
    }

    results = tune.Tuner(
        args.run,
        run_config=air.RunConfig(stop=stop, verbose=2),
        param_space=config,
    ).fit()

    if args.as_test:
        check_learning_achieved(results, args.stop_reward)

    ray.shutdown()

2023-07-06 11:18:39,415	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(pid=49584) Windows fatal exception: code 0xc0000139
(pid=49584) 
(pid=49584) Windows fatal exception: code 0xc0000139
(pid=49584) 
(pid=49584) Windows fatal exception: code 0xc0000139
(pid=49584) 
(pid=49584) Windows fatal exception: code 0xc0000139
(pid=49584) 
(pid=49584) Windows fatal exception: code 0xc0000139
(pid=49584) 
(pid=49584) Windows fatal exception: code 0xc0000139
(pid=49584) 
(pid=49584) Windows fatal exception: code 0xc0000139
(pid=49584) 
(pid=49584) Windows fatal exception: code 0xc0000139
(pid=49584) 
(pid=49584) C:\Users\subar\anaconda3\envs\d\lib\site-packages\gym\envs\registration.py:307: DeprecationWarning: The package name gym_minigrid has been deprecated in favor of minigrid. Please uninstall gym_minigrid and install minigrid with `pip install minigrid`. Future releases will be maintained under the new package name minigrid.
(pid=49584)   fn()
(pid=49584) Windows fatal exception: code 0xc0000139
(pid=49584) 
(pid=49584) Windows fatal exception: code 0xc000013

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,timers
QMIX_grouped_twostep_89a90_00000,70000,"{'ObsPreprocessorConnector_ms': 0.09924001693725586, 'StateBufferConnector_ms': 0.018151473999023438, 'ViewRequirementAgentConnector_ms': 0.25716204643249513}","{'num_env_steps_sampled': 70000, 'num_env_steps_trained': 552000, 'num_agent_steps_sampled': 70000, 'num_agent_steps_trained': 552000, 'last_target_update_ts': 69504, 'num_target_updates': 138}",{},2,{},7,7,7,500,"{'learner': {'default_policy': {'learner_stats': {'loss': 7.590557652292773e-05, 'td_error_abs': 0.008675613440573215, 'q_taken_mean': 0.010148399509489536, 'target_mean': 0.0014727864181622863, 'grad_gnorm': 0.0645175576210022}}}, 'num_env_steps_sampled': 70000, 'num_env_steps_trained': 552000, 'num_agent_steps_sampled': 70000, 'num_agent_steps_trained': 552000, 'last_target_update_ts': 69504, 'num_target_updates': 138}",70000,552000,70000,1000,552000,8000,0,0,0,0,8000,"{'cpu_util_percent': 7.3, 'ram_util_percent': 51.79999999999999, 'gpu_util_percent0': 0.32499999999999996, 'vram_util_percent0': 0.1719970703125}",{},{},{},"{'mean_raw_obs_processing_ms': 0.9526090978481107, 'mean_inference_ms': 0.7371998581194408, 'mean_action_processing_ms': 0.15473369189622452, 'mean_env_wait_ms': 0.06172967667910093, 'mean_env_render_ms': 0.0}","{'training_iteration_time_ms': 16.485, 'learn_time_ms': 6.483, 'learn_throughput': 4936.345, 'synch_weights_time_ms': 0.0}"


2023-07-06 11:23:21,620	INFO tune.py:798 -- Total run time: 279.24 seconds (278.57 seconds for the tuning loop).


In [24]:
ray.shutdown()

In [22]:
def generate_rollout_from_current_trainer_policy(
    trainer, 
    env_obj,
    num_dense_logs=1
):
    dense_logs = {}
    for idx in range(num_dense_logs):
        # Set initial states
        agent_states = {}
        for agent_idx in range(3000):
            agent_states[str(agent_idx)] = trainer.get_policy("high_level_policy").get_initial_state()
        planner_states = trainer.get_policy("group_1").get_initial_state()   
        """
        # Get weights of the default local policy
        algo.get_policy().get_weights()

        # Same as above
        algo.workers.local_worker().policy_map["default_policy"].get_weights()

        # Get list of weights of each worker, including remote replicas
        algo.workers.foreach_worker(lambda worker: worker.get_policy().get_weights())

        # Same as above, but with index.
        algo.workers.foreach_worker_with_id(
            lambda _id, worker: worker.get_policy().get_weights()
        )
        
        
        """
        # Play out the episode
        obs = env_obj.reset(force_dense_logging=True)
        for t in range(env_obj.episode_length):
            actions = {}
            for agent_idx in range(2):
                # Use the trainer object directly to sample actions for each agent
                actions[str(agent_idx)] = trainer.compute_action(
                    obs[str(agent_idx)], 
                    agent_states[str(agent_idx)], 
                    policy_id="high_level_policy",
                    full_fetch=False
                )

            # Action sampling for the planner
            actions["group_1"] = trainer.compute_action(
                obs['group_1'], 
                planner_states, 
                policy_id='low_level_policy',
                full_fetch=False
            )

            obs, rew, done, info = env_obj.step(actions)        
            if done['__all__']:
                break
        dense_logs[idx] = env_obj.dense_log

In [22]:
from ray.air.config import RunConfig, ScalingConfig
from ray.train.rl import RLTrainer
from gymnasium.spaces import Dict, Discrete, MultiDiscrete, Tuple,Box

logger = logging.getLogger(__name__)

parser = argparse.ArgumentParser()
parser.add_argument(
    "--run", type=str, default="QMIX", help="The RLlib-registered algorithm to use."
)
parser.add_argument(
    "--framework",
    choices=["tf", "tf2", "torch"],
    default="torch",
    help="The DL framework specifier.",
)
parser.add_argument("--num-cpus", type=int, default=16)
#parser.add_argument("--num-gpus", type=int, default=1)
parser.add_argument("--num-gpus", type=float, default=0)#0.25)

parser.add_argument("--num-workers", type=int, default=6)
parser.add_argument("--num-gpus-per-worker", type=float, default=0)
#parser.add_argument("render_mode", type=int, default=1)
parser.add_argument(
    "--mixer",
    type=str,
    default="qmix",
    choices=["qmix", "vdn", "none"],
    help="The mixer model to use.",
)
parser.add_argument(
    "--as-test",
    action="store_true",
    help="Whether this script should be run as a test: --stop-reward must "
    "be achieved within --stop-timesteps AND --stop-iters.",
)
parser.add_argument(
    "--stop-iters", type=int, default=200, help="Number of iterations to train."
)
parser.add_argument(
    "--stop-timesteps", type=int, default=70000, help="Number of timesteps to train."
)
parser.add_argument(
    "--stop-reward", type=float, default=9.0, help="Reward at which we stop training."
)
parser.add_argument(
    "--local-mode",
    action="store_true",
    help="Init Ray in local mode for easier debugging.",
)
if __name__ == "__main__":
    
    #def policy_mapping_fn(agent_id, episode, worker, **kwargs):
            #if agent_id.startswith("low_level_"):
                #return "low_level_policy"
            #else:
                #return "high_level_policy"
    args = parser.parse_args()

    ray.init(num_cpus=16, num_gpus=1,local_mode=args.local_mode)
    
    """
    we are going to first experiment with determining whether the interactions between the agents in group 1
    can be captured and visualized. An agent group is a list of agent IDs that are mapped to a single
    logical agent. All these agents must act at the same time 
    
    
    """
    
    grouping = {
        "group_1": [0,1],
    }
    def policy_mapping_fn(agent_id, episode, worker, **kwargs):
        #if agent_id.startswith("low_level_"):
        if agent_id.startswith(0):
            return "low_level_policy"
        else:
            return "high_level_policy"
    obs_space = Tuple(
        [
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
            Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
        ]
    )
    act_space = Tuple(
        [
            Discrete(2),#TwoStepGame.action_space,
            Discrete(2),#TwoStepGame.action_space,
        ]
    )
    register_env(
        "grouped_twostep",
        lambda config: TwoStepGame(config).with_agent_groups(
            grouping, obs_space=obs_space, act_space=act_space
        ),
    )
    """
    grouping = {
        "group_1": [0],
        "group_2": [0,1],#2
        #"group_3": [0]
    }


    
    from ray.tune import register_env
    from ray.rllib.algorithms.dqn import DQN 
    YourExternalEnv = ... 
    register_env("my_env", 
        lambda config: YourExternalEnv(config))
    trainer = DQN(env="my_env") 
    while True: 
        print(trainer.train()) 

    """


    config = (
        get_trainable_cls(args.run)
        .get_default_config()
        .environment(TwoStepGame)#
        .framework(args.framework)
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=args.num_gpus,num_gpus_per_worker=args.num_gpus_per_worker,)
        #.reset_config(reuse_actors=True)
    )
    if args.run == "MADDPG":
        obs_space = Discrete(6)
        act_space = TwoStepGame.action_space
        (
            config.framework("tf")
            .environment(env_config={"actions_are_logits": True})
            .training(num_steps_sampled_before_learning_starts=100)
            .multi_agent(
                policies={
                    "high_level_policy": PolicySpec(
                        observation_space=obs_space,
                        action_space=act_space,
                        config=config.overrides(agent_id=0),
                    ),
                    "low_level_policy": PolicySpec(
                        observation_space=obs_space,
                        action_space=act_space,
                        config=config.overrides(agent_id=1),
                    ),
                },
                policy_mapping_fn=policy_mapping_fn#lambda agent_id, episode, worker, **kwargs: "pol2"
                #if agent_id
                #else "pol1",
            )
        )
    elif args.run == "QMIX":
        (
        config.framework("torch")
        .training(mixer=args.mixer, train_batch_size=32)
            
        .rollouts(rollout_fragment_length=4)
        .exploration(
            exploration_config={
                "final_epsilon": 0.0,
            }
        )
        .environment(
            env="grouped_twostep",
            env_config={
                "separate_state_space": True,
                "one_hot_state_encoding": True,
            },
        )
    )

    stop = {
        "episode_reward_mean": args.stop_reward,
        "timesteps_total": args.stop_timesteps,
        "training_iteration": args.stop_iters,
    }
    from ray.tune.schedulers import PopulationBasedTraining

    pbt_scheduler = PopulationBasedTraining(
        time_attr='training_iteration',
        metric='episode_reward_mean',#'loss',
        mode='min',
        perturbation_interval=1,
        hyperparam_mutations={
            #"lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
            "alpha": tune.uniform(0.0, 1.0),
        }
    )
    trainer = RLTrainer(
        "QMIX",#args.run,
        run_config=air.RunConfig(stop=stop, verbose=2),#RunConfig(stop={"training_iteration": 5}),
        scaling_config=ScalingConfig(num_workers=10,use_gpu=True),
        #algorithm="QMIX",
        config=config.to_dict()
        
    ).fit()
    traine = RLTrainer(
        args.run,
        run_config=air.RunConfig(stop=stop, verbose=2),#RunConfig(stop={"training_iteration": 5}),
        scaling_config=ScalingConfig(num_workers=10,use_gpu=True),
        #algorithm="QMIX",
        config=config.to_dict()
        
    )
    #con = config.to_dict()
    dense_logs = {}
    num_dense_logs=1
    #args.run.workers.foreach_worker(lambda worker: worker.get_policy().get_weights())
    #import rllib
    #conf = config.build()
    #env_obj = env
    #env = config#TwoStepGame(config)
    #env_obj = con.build()#RLlibEnvWrapper({"env_config_dict": con}, verbose=True)
    

    """
    algo.get_policy("high_level_policy").get_weights()
    #QMixConfig()
    # Same as above
    
    # Same as above
    algo.workers.local_worker().policy_map["high_level_policy"].get_weights()

    # Get list of weights of each worker, including remote replicas
    algo.workers.foreach_worker(lambda worker: worker.get_policy().get_weights())

    # Same as above, but with index.
    algo.workers.foreach_worker_with_id(
        lambda _id, worker: worker.get_policy().get_weights()
    )
    
    algo.workers.local_worker().policy_map["low_level_policy"].get_weights()

    # Get list of weights of each worker, including remote replicas
    algo.workers.foreach_worker(lambda worker: worker.get_policy().get_weights())

    # Same as above, but with index.
    algo.workers.foreach_worker_with_id(
        lambda _id, worker: worker.get_policy().get_weights()
    )
    
    # Get weights of the default local policy
    algo.get_policy().get_weights()

    # Same as above
    algo.workers.local_worker().policy_map["default_policy"].get_weights()

    # Get list of weights of each worker, including remote replicas
    algo.workers.foreach_worker(lambda worker: worker.get_policy().get_weights())

    # Same as above, but with index.
    algo.workers.foreach_worker_with_id(
        lambda _id, worker: worker.get_policy().get_weights()
    )
    
    """
    #alg = args.run.build()
    dense_logs = generate_rollout_from_current_trainer_policy(
    trainer, 
    env_obj=alg,
    num_dense_logs=2)

    #result = trainer.fit().compute_actions_from_input_dict(_

2023-08-01 01:01:12,274	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 


(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) Windows fatal exception: code 0xc0000139
(pid=37932) 
(pid=37932) C:\Users\subar\anaconda3\envs\d\lib\site-packages\flax\struct.py

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,timers
AIRQMix_96ced_00000,70040,"{'ObsPreprocessorConnector_ms': 0.21562061309814454, 'StateBufferConnector_ms': 0.04076075553894043, 'ViewRequirementAgentConnector_ms': 0.5932717800140381}","{'num_env_steps_sampled': 70040, 'num_env_steps_trained': 55232, 'num_agent_steps_sampled': 70040, 'num_agent_steps_trained': 55232, 'last_target_update_ts': 69680, 'num_target_updates': 133}",{},2,{},8,4.48,0,500,"{'learner': {'default_policy': {'learner_stats': {'loss': 0.0010980626102536917, 'td_error_abs': 0.023351505398750305, 'q_taken_mean': 0.10706064105033875, 'target_mean': 0.08690129220485687, 'grad_gnorm': 0.27305924892425537}}}, 'num_env_steps_sampled': 70040, 'num_env_steps_trained': 55232, 'num_agent_steps_sampled': 70040, 'num_agent_steps_trained': 55232, 'last_target_update_ts': 69680, 'num_target_updates': 133}",70040,55232,70040,1000,55232,800,0,10,0,0,800,"{'cpu_util_percent': 22.4, 'ram_util_percent': 62.8, 'gpu_util_percent0': 0.15, 'vram_util_percent0': 0.1490478515625}",{},{},{},"{'mean_raw_obs_processing_ms': 2.2128118369342773, 'mean_inference_ms': 1.4947364808490327, 'mean_action_processing_ms': 0.328442898926609, 'mean_env_wait_ms': 0.11809317066701797, 'mean_env_render_ms': 0.0}","{'training_iteration_time_ms': 85.981, 'learn_time_ms': 25.621, 'learn_throughput': 1248.983, 'synch_weights_time_ms': 24.052}"


2023-08-01 01:04:46,671	INFO tune.py:798 -- Total run time: 211.06 seconds (210.84 seconds for the tuning loop).


NameError: name 'generate_rollout_from_current_trainer_policy' is not defined

In [21]:
ray.shutdown()

In [116]:
register_env(
    "grouped_twoste",
    lambda config: TwoStepGame(config).with_agent_groups({"group1": ["agent1", "agent2", "agent3"],"group2": ["agent4", "agent5"]})#,obs_space={obs_space,obs_space})) 

In [20]:
import argparse
from gymnasium.spaces import Dict, Discrete, Tuple, MultiDiscrete
import logging
import os

import ray
from ray import air, tune
from ray.tune import register_env
from ray.rllib.env.multi_agent_env import ENV_STATE
from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import get_trainable_cls


In [ ]:
"""
in this set up only the top level is trained and goals are conveyed from top to bottom

For our thing we need to train the bottem layer and the top layer create custom policies fro the top and 
bottem layers 

the bottem layer will be based on 

"""

"multiagent": {
    "policies": {
        "top_level": (custom_policy or None, ...),
        "mid_level": (custom_policy or None, ...),
        "low_level": (custom_policy or None, ...),
    },
    "policy_mapping_fn":
        lambda agent_id:
            "low_level" if agent_id.startswith("low_level_") else
            "mid_level" if agent_id.startswith("mid_level_") else "top_level"
    "policies_to_train": ["top_level"],
},

In [21]:
#from ray.rllib.examples.env.two_step_game import TwoStepGame
from gymnasium.spaces import Dict, Discrete, MultiDiscrete, Tuple,Box
"""
 grouped_env = env.with_agent_groups(env, { # doctest: +SKIP
            ...   "group1": ["agent1", "agent2", "agent3"], # doctest: +SKIP
            ...   "group2": ["agent4", "agent5"], # doctest: +SKIP
            ... }) 

"""
"""
we are just going to try it out with 3 agents in a twostep game example and do a visualiaation there

Later we are going to have 3 groupings and two of the grouping will be on level 1 and 1 grouping will be on 
level 0 

"""
grouping = {"pol2": [0, 1], "pol1": [2]}
obs_space = Discrete(6)
act_space = TwoStepGame.action_space
register_env(
    "grouped_twostep",
    lambda config: TwoStepGame(config).with_agent_groups(
        grouping, obs_space=obs_space, act_space=act_space
    ),
)

obs_space = Tuple(
    [
        Dict({"obs": MultiDiscrete([2, 2, 2, 3]), ENV_STATE: MultiDiscrete([2, 2, 2])}),
        Dict({"obs": MultiDiscrete([2, 2, 2, 3]), ENV_STATE: MultiDiscrete([2, 2, 2])}),
    ]
)
act_space = Tuple([TwoStepGame.action_space, TwoStepGame.action_space])
def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    #if agent_id.startswith("low_level_"):
    if agent_id != 2:#.startswith(0):
        return "pol2"
    else:
        return "pol1"
algo= (
    get_trainable_cls('QMIX')
    .get_default_config()
    .environment(
        env="grouped_twostep",#TwoStepGameWithGroupedAgents",
        env_config={
            "separate_state_space": True,
            "one_hot_state_encoding": True,
        },
    )
    
    #.environment('grouped_twostep')#
    .framework('torch')
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    .resources(num_gpus=1,num_gpus_per_worker=0,)
    #.reset_config(reuse_actors=True)
    .multi_agent(
        policies={
            "pol1": PolicySpec(
                        observation_space=obs_space,
                        action_space=act_space,
                        #config=config.overrides(agent_id=0),
            ),
            "pol2": PolicySpec(
                        observation_space=obs_space,
                        action_space=act_space,
                        #config=config.overrides(agent_id=1),
            ),
        },
        policy_mapping_fn=policy_mapping_fn#lambda agent_id, episode, worker, **kwargs: "pol2"if agent_id else "pol1",#

    )
)



In [22]:
algo = algo.build()
print(algo)
for _ in range(2):
    #print(algo.get_policy("pol1"))
    print(algo.get_policy("pol2"))
    print(algo.get_weights())
    algo.train()  # 3. train it,

2023-08-05 13:05:18,335	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 
2023-08-05 13:05:20,762	WARNING env.py:53 -- Skipping env checking for this experiment
2023-08-05 13:05:23,546	INFO trainable.py:172 -- Trainable.setup took 10.281 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


QMix
QMixTorchPolicy
{'pol1': {'model': {'fc1.weight': array([[-0.2760169 ,  0.21935067, -0.2124548 , -0.07207838,  0.19570842,
        -0.12978148, -0.19238476,  0.23792687, -0.06051981],
       [-0.1623383 , -0.09331279,  0.08181503, -0.042714  , -0.2517395 ,
         0.17846075, -0.07867411, -0.02682519, -0.2696974 ],
       [-0.13755207, -0.01649752, -0.20989689, -0.26931155, -0.31673682,
         0.3002253 ,  0.16486552,  0.09260166, -0.06003866],
       [-0.12329838, -0.2997555 , -0.21749783, -0.01876268,  0.15480351,
         0.12033775,  0.07103992, -0.25473458,  0.08300805],
       [-0.06317925, -0.3157518 ,  0.2136732 , -0.02455196, -0.29533705,
         0.2867979 , -0.14578879,  0.2359716 , -0.15744905],
       [ 0.16477537, -0.09850399, -0.32760534,  0.07474473,  0.02643907,
        -0.1743598 ,  0.09386727,  0.12847063, -0.12036566],
       [ 0.29764065,  0.24053577, -0.28303108,  0.05418244, -0.0664897 ,
        -0.15922639, -0.27148002, -0.18399572,  0.12165773],
       

In [29]:
#algo.get_policy().get_weights()

# Same as above
algo.workers.local_worker().policy_map["pol1"].get_weights()

# Get list of weights of each worker, including remote replicas
algo.workers.foreach_worker(lambda worker: worker.get_policy("pol2").get_weights())

# Same as above, but with index.
algo.workers.foreach_worker_with_id(
    lambda _id, worker: worker.get_policy("pol1").get_weights()
)

[{'model': {'fc1.weight': array([[-0.2760169 ,  0.21935067, -0.2124548 , -0.07207838,  0.19570842,
           -0.12978148, -0.19238476,  0.23792687, -0.06051981],
          [-0.1623383 , -0.09331279,  0.08181503, -0.042714  , -0.2517395 ,
            0.17846075, -0.07867411, -0.02682519, -0.2696974 ],
          [-0.13755207, -0.01649752, -0.20989689, -0.26931155, -0.31673682,
            0.3002253 ,  0.16486552,  0.09260166, -0.06003866],
          [-0.12329838, -0.2997555 , -0.21749783, -0.01876268,  0.15480351,
            0.12033775,  0.07103992, -0.25473458,  0.08300805],
          [-0.06317925, -0.3157518 ,  0.2136732 , -0.02455196, -0.29533705,
            0.2867979 , -0.14578879,  0.2359716 , -0.15744905],
          [ 0.16477537, -0.09850399, -0.32760534,  0.07474473,  0.02643907,
           -0.1743598 ,  0.09386727,  0.12847063, -0.12036566],
          [ 0.29764065,  0.24053577, -0.28303108,  0.05418244, -0.0664897 ,
           -0.15922639, -0.27148002, -0.18399572,  0.12165773

In [71]:
#config = {}
a = TwoStepGame(config).with_agent_groups(
            grouping, obs_space=obs_space, act_space=act_space)

In [ ]:
print(a.reset())#initial observations
obs,info = a.reset()

print(obs)
print(info)
a.step(actions=obs)
print(algo.get_policy().compute_actions())#compute_single_action(obs))

In [62]:
from ray.rllib.examples.env.two_step_game import TwoStepGame
from gymnasium.spaces import Dict, Discrete, MultiDiscrete, Tuple,Box
import argparse
import logging
import os

import ray
from ray import air, tune
from ray.tune import register_env
from ray.rllib.env.multi_agent_env import ENV_STATE
from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import get_trainable_cls

In [63]:
import argparse
from gymnasium.spaces import Dict, Discrete, Tuple, MultiDiscrete
import logging
import os

import ray
from ray import air, tune
from ray.tune import register_env
from ray.rllib.env.multi_agent_env import ENV_STATE
from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import get_trainable_cls

In [ ]:
config = {}
a = TwoStepGame(config).with_agent_groups(
            grouping, obs_space=obs_space, act_space=act_space)

In [68]:

grouping = {"group_1": [0, 1], "group_2": [2]}
obs_space = Tuple(
    [
        Dict({"obs": MultiDiscrete([2, 2, 2, 3]), ENV_STATE: MultiDiscrete([2, 2, 2])}),
        Dict({"obs": MultiDiscrete([2, 2, 2, 3]), ENV_STATE: MultiDiscrete([2, 2, 2])}),
    ]
)
act_space = Tuple([TwoStepGame.action_space, TwoStepGame.action_space])
register_env(
    "grouped_twostep",
    lambda config: TwoStepGame(config).with_agent_groups(
        grouping, obs_space=obs_space, act_space=act_space
    ),
)
"""
config = (
    get_trainable_cls('QMIX')
    .get_default_config()
    .environment(
        env="grouped_twoste",#TwoStepGameWithGroupedAgents",
        env_config={
            "separate_state_space": True,
            "one_hot_state_encoding": True,
        },
    )
    
    #.environment('grouped_twostep')#
    .framework('torch')
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    .resources(num_gpus=1,num_gpus_per_worker=0,)
    #.reset_config(reuse_actors=True)
    .multi_agent(
        policies={
            "pol1": PolicySpec(
                obs_space=obs_space,
                act_space=act_space,
                config=config.overrides(agent_id=0),
            ),
            "pol2": PolicySpec(
                obs_space=obs_space,
                act_space=act_space,
                config=config.overrides(agent_id=1),
            ),
        },
        policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "pol2"
        if agent_id
        else "pol1",
    )
)
"""
config = {
    
    "env": "grouped_twostep",
    "multiagent": {
        "policies": {
            "pol1": (None, obs_space, act_space, {}),
            "pol2": (None, obs_space, act_space, {}),
        },
        "policy_mapping_fn": lambda agent_id, episode, worker, **kwargs: "pol2"
        if agent_id
        else "pol1",#lambda x: "pol1" if cond else "pol2",
    },
}
algo = config.build()
print(algo)
for _ in range(2):
    print(algo.get_policy(0))
    algo.train()  # 3. train it,


AttributeError: 'dict' object has no attribute 'build'

In [64]:
config = {
    "--run": "QMIX",
    "env": "grouped_twostep",
    "multiagent": {
        "policies": {
            "pol1": (None, obs_space, act_space, {}),
            "pol2": (None, obs_space, act_space, {}),
        },
        "policy_mapping_fn": lambda x: "pol1" if cond else "pol2",
    },
}

algo = config.build()
print(algo)
for _ in range(2):
    print(algo.get_policy(0))
    algo.train()  # 3. train it,

AttributeError: 'dict' object has no attribute 'build'

In [ ]:
tune.run("QMIX", config=config)

In [67]:
for _ in range(2):
    print(algo.train())  # 3. train it,

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'pol2': {'learner_stats': {'loss': 0.0011464310809969902, 'td_error_abs': 0.0316702239215374, 'q_taken_mean': 0.22500228881835938, 'target_mean': 0.19333206117153168, 'grad_gnorm': 0.5485808253288269}}}, 'num_env_steps_sampled': 3000, 'num_env_steps_trained': 16000, 'num_agent_steps_sampled': 3000, 'num_agent_steps_trained': 16000, 'last_target_update_ts': 2504, 'num_target_updates': 4}, 'sampler_results': {'episode_reward_max': 8.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 4.686, 'episode_len_mean': 2.0, 'episode_media': {}, 'episodes_this_iter': 500, 'policy_reward_min': {'pol2': 0.0}, 'policy_reward_max': {'pol2': 8.0}, 'policy_reward_mean': {'pol2': 4.686}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [1.0, 0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 1.0, 7.0, 7.0, 7.0, 7.0, 8.0, 8.0, 7.0, 1.0, 8.0, 7.0, 0.0, 0.0, 7.0, 1.0, 1.0, 7.0, 8.0, 7.0, 7.0, 8.0, 1.0, 7.0, 1.0, 7.0, 7.0, 7.0, 0.0, 7.0, 1.0, 1.0, 

In [34]:
algo.get_policy().get_weights()

{'_logits._model.0.weight': array([[ 6.63691061e-03, -9.86987841e-04,  3.42994183e-03,
         -4.57203249e-03, -1.15657528e-03,  4.48928848e-02,
          6.82245344e-02,  2.65770089e-02,  5.42110838e-02,
         -1.92943532e-02,  8.42820387e-03, -3.14753912e-02,
         -6.18160143e-02, -7.09734932e-02, -5.31210825e-02,
         -3.74888293e-02,  2.82458868e-02,  2.20551249e-02,
          8.78330767e-02, -4.45778370e-02, -2.52658725e-02,
         -5.31081520e-02,  5.23066185e-02, -7.22493082e-02,
          6.36454746e-02,  4.82033119e-02,  6.58804625e-02,
         -3.59875057e-03,  9.45299026e-03, -2.44920347e-02,
          3.86952348e-02, -5.43356054e-02, -2.22519273e-03,
          1.20062707e-02,  5.04499376e-02,  3.79170738e-02,
         -2.64225900e-02,  6.68922961e-02,  5.61541095e-02,
          6.82325512e-02, -4.65018414e-02, -3.46434638e-02,
         -5.38668595e-02,  5.14229201e-02,  4.37635519e-02,
          1.23130996e-02,  4.29462083e-02, -7.46519342e-02,
          4.8

In [25]:
def generate_rollout_from_current_trainer_policy(
    trainer, 
    env_obj,
    num_dense_logs=1
):
    dense_logs = {}
    for idx in range(num_dense_logs):
        # Set initial states
        agent_states = {}
        for agent_idx in range(3000):
            agent_states[str(agent_idx)] = trainer.get_policy("high_level_policy").get_initial_state()
        planner_states = trainer.get_policy("group_1").get_initial_state()   
        """
        # Get weights of the default local policy
        algo.get_policy().get_weights()

        # Same as above
        algo.workers.local_worker().policy_map["default_policy"].get_weights()

        # Get list of weights of each worker, including remote replicas
        algo.workers.foreach_worker(lambda worker: worker.get_policy().get_weights())

        # Same as above, but with index.
        algo.workers.foreach_worker_with_id(
            lambda _id, worker: worker.get_policy().get_weights()
        )
        
        
        """
        # Play out the episode
        obs = env_obj.reset(force_dense_logging=True)
        for t in range(env_obj.episode_length):
            actions = {}
            for agent_idx in range(2):
                # Use the trainer object directly to sample actions for each agent
                actions[str(agent_idx)] = trainer.compute_action(
                    obs[str(agent_idx)], 
                    agent_states[str(agent_idx)], 
                    policy_id="high_level_policy",
                    full_fetch=False
                )

            # Action sampling for the planner
            actions["group_1"] = trainer.compute_action(
                obs['group_1'], 
                planner_states, 
                policy_id='high_level_policy',
                full_fetch=False
            )

            obs, rew, done, info = env_obj.step(actions)        
            if done['__all__']:
                break
        dense_logs[idx] = env_obj.dense_log

In [26]:
dense_logs = generate_rollout_from_current_trainer_policy(
    trainer, 
    'grouped_twostep',
    num_dense_logs=2
)

AttributeError: 'Result' object has no attribute 'get_policy'

In [76]:
ray.shutdown()

In [ ]:
print(dense_logs)

In [52]:
#algo = get_trainable_cls(args.run)#.build()##args.run#TwoStepGame(config)
#print(algo)
#print(config)
#alg = config.build()
#algo.get_policy('low_level')#.get_weights()
#print(alg)
#print(traine.policy('low_level_policy'))
#alg.workers.foreach_worker(lambda worker: worker.get_policy('low_level_policy').get_weights())
# Same as above, but with index.

algo.workers.foreach_worker_with_id(
    lambda _id, worker: worker.get_policy("").get_weights()
)


AttributeError: 'NoneType' object has no attribute 'get_weights'

In [41]:
for i in range(10):
    print(alg.train()) 
    #env = TwoStepGame(config)
    obs, info = env.reset()
    print(obs)
    print(info)
    
    
    """
    action = algo.compute_single_action(#trainer.compute_action
                obs[1],  
                policy_id='low_level_policy',
                full_fetch=False
            )#algo.compute_single_action(3)
    print(action)
    """
    
    obs, reward, terminated, truncated, info = env.step(0)

    #Policy.compute_actions_from_input_dict()

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'policy_loss': 1.8010945916175842, 'cur_lr': 0.0004}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 200.0, 'num_grad_updates_lifetime': 1.5, 'diff_num_grad_updates_vs_sampler_policy': 0.5}}, 'num_env_steps_sampled': 200, 'num_env_steps_trained': 200, 'num_agent_steps_sampled': 400, 'num_agent_steps_trained': 400}, 'sampler_results': {'episode_reward_max': 8.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 5.24, 'episode_len_mean': 2.0, 'episode_media': {}, 'episodes_this_iter': 100, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [7.0, 7.0, 8.0, 7.0, 7.0, 7.0, 1.0, 7.0, 0.0, 1.0, 8.0, 7.0, 1.0, 8.0, 7.0, 7.0, 7.0, 1.0, 7.0, 7.0, 7.0, 7.0, 8.0, 8.0, 8.0, 0.0, 7.0, 7.0, 7.0, 8.0, 1.0, 1.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 1.0, 1.0, 1.0, 7.0, 0.0, 7.0, 0.0, 8.

NameError: name 'env' is not defined

In [53]:
obs, reward, terminated, truncated, info = env.step()
print(obs)

NameError: name 'env' is not defined

In [40]:
algo = get_trainable_cls(args.run)#.build()##args.run#TwoStepGame(config)
print(algo)
alg = config.build()
#b = traine.get_policy('high_level')#.get_weights()
print(alg)
#print(b)
c = alg.workers.foreach_worker(lambda worker: worker.get_policy().get_weights())
#print(trainer.get_policy("low_level"))
print(c)
c = np.array(c)
print(c.shape)
"""
obs = env.reset()
Then, we call step to advance the state and advance time by one tick.

actions = sample_random_actions(env, obs)
obs, rew, done, info = env.step(actions)


"""

<class 'ray.rllib.algorithms.pg.pg.PG'>
PG
[{'one_hot.0._hidden_layers.0._model.0.weight': array([[-0.2417314 ,  0.13188548, -0.11307124,  0.65049994, -0.5958364 ,
        -0.36498737],
       [-0.15053855,  0.34130767,  0.6101184 , -0.6719392 , -0.09989117,
        -0.16468833],
       [ 0.06607527,  0.53549373, -0.138774  , -0.08091102, -0.8257553 ,
         0.03469821],
       ...,
       [-0.6290907 ,  0.07368284, -0.02462804, -0.65478146,  0.03681446,
         0.4100185 ],
       [ 0.48683763, -0.47035825,  0.5672494 , -0.0302962 , -0.46104297,
         0.0806331 ],
       [-0.4187051 ,  0.23055151, -0.47441444, -0.10279853, -0.35409957,
         0.64070976]], dtype=float32), 'one_hot.0._hidden_layers.0._model.0.bias': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.

'\nobs = env.reset()\nThen, we call step to advance the state and advance time by one tick.\n\nactions = sample_random_actions(env, obs)\nobs, rew, done, info = env.step(actions)\n\n\n'

In [98]:
algo = config.build()
"""
as of 6/19/2023 this is no longer the best



"""
print(algo)

# Train.
for i in range(10):
    print(algo.train()) 
    env = TwoStepGame(config)
    obs, info = env.reset()
    print(obs)
    print(info)
    """
    action = algo.compute_single_action(#trainer.compute_action
                obs[1],  
                policy_id='low_level_policy',
                full_fetch=False
            )#algo.compute_single_action(3)
    print(action)
    """
    
    obs, reward, terminated, truncated, info = env.step(0)
#ray.shutdown()


PG
{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'policy_loss': 1.645327866077423, 'cur_lr': 0.0004}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 200.0, 'num_grad_updates_lifetime': 1.5, 'diff_num_grad_updates_vs_sampler_policy': 0.5}}, 'num_env_steps_sampled': 200, 'num_env_steps_trained': 200, 'num_agent_steps_sampled': 400, 'num_agent_steps_trained': 400}, 'sampler_results': {'episode_reward_max': 8.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 4.78, 'episode_len_mean': 2.0, 'episode_media': {}, 'episodes_this_iter': 100, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [7.0, 1.0, 7.0, 7.0, 8.0, 0.0, 7.0, 7.0, 1.0, 0.0, 7.0, 8.0, 1.0, 7.0, 1.0, 8.0, 7.0, 7.0, 0.0, 1.0, 7.0, 1.0, 7.0, 0.0, 7.0, 7.0, 8.0, 1.0, 7.0, 7.0, 7.0, 0.0, 8.0, 1.0, 7.0, 7.0, 7.0, 7.0, 1.0, 1.0, 1.0, 7.0, 8.0, 8.0, 8.0, 7.0, 

KeyError: "PolicyID 'low_level_policy' not found in PolicyMap of the Trainer's local worker!"

In [46]:
ray.shutdown()

In [ ]:
#offline learning

import ray
from ray.air.config import RunConfig, ScalingConfig
from ray.train.rl import RLTrainer
from ray.rllib.algorithms.bc.bc import BC
"""
dataset = ray.data.read_json(
    "/tmp/data-dir", parallelism=2, ray_remote_args={"num_cpus": 1}
)

trainer = RLTrainer(
    run_config=RunConfig(stop={"training_iteration": 5}),
    scaling_config=ScalingConfig(
        num_workers=2,
        use_gpu=False,
    ),
    datasets={"train": dataset},
    algorithm=BCTrainer,
    config={
        "env": "CartPole-v0",
        "framework": "tf",
        "evaluation_num_workers": 1,
        "evaluation_interval": 1,
        "evaluation_config": {"input": "sampler"},
    },
)
result = trainer.fit()

"""


In [28]:
#replay buffer reference
algo = config.build()

# Train.
for i in range(1):
    algo.train()
    #buffer = ReplayBuffer(capacity=2, storage_unit=StorageUnit.FRAGMENTS)
    buffer = MultiAgentPrioritizedReplayBuffer(capacity=2,storage_unit='timesteps')#MultiAgentPrioritizedReplayBuffer((capacity=2)
    
    print(algo.workers.foreach_worker_with_id(
        lambda _id, worker: worker.previous_episode_dense_log()
    ))
    
    #sample_batch = buffer.sample(1)
    #dummy_batch = SampleBatch({"a": [1], "b": [2]})
    #buffer.add(dummy_batch)
    print("                sample                ")
    print()
    #print(sample_batch.MultiAgentBatch("low_level_policy"))
    #print(sample_min_n_steps_from_buffer(buffer,2,True))
    
    """
    new_obs, rewards, dones, infos = env.step()
    
    """
    
    #print(buffer.sample(2))
    #print(ray.rllib.utils.replay_buffers.utils.sample_min_n_steps_from_buffer(MultiAgentPrioritizedReplayBuffer,5,True))
    #ray.rllib.utils.replay_buffers.utils.sample_min_n_steps_from_buffer(
    
    #obs, reward, terminated, truncated, info = TwoStepGame.step()
    # Note that in order for RLlib to find out about start and end of an episode,
    # "t" and "terminateds" have to properly mark an episode's trajectory

    print("####################################")

ray.shutdown()


2023-06-12 15:26:30,079	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-06-12 15:26:53,821	INFO trainable.py:172 -- Trainable.setup took 30.595 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


AttributeError: 'RolloutWorker' object has no attribute 'previous_episode_dense_log'

In [23]:
# Below, we fetch the dense logs for each rollout worker and environment within

dense_logs = {}
# Note: worker 0 is reserved for the trainer actor
for worker in range((trainer_config["num_workers"] > 0), trainer_config["num_workers"] + 1):
    for env_id in range(trainer_config["num_envs_per_worker"]):
        dense_logs["worker={};env_id={}".format(worker, env_id)] = \
        trainer.workers.foreach_worker(lambda w: w.async_env)[worker].envs[env_id].env.previous_episode_dense_log

NameError: name 'trainer_config' is not defined

In [34]:
print(algo.evaluate()) 

2023-06-11 01:53:17,196	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


{'evaluation': {'episode_reward_max': 8.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 6.605, 'episode_len_mean': 2.0, 'episode_media': {}, 'episodes_this_iter': 1000, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [7.0, 1.0, 7.0, 7.0, 7.0, 7.0, 8.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 8.0, 8.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 7.0, 7.0, 8.0, 1.0, 7.0, 8.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 8.0, 7.0, 1.0, 7.0, 7.0, 1.0, 7.0, 1.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 8.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 8.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 0.0, 7.0, 8.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0,

In [64]:
algo = config.build()

# Train.
for i in range(10):
    print(algo.train())
    
    obs, info = env.reset()
    
    action = algo.compute_single_action(obs)
    obs, reward, terminated, truncated, info = env.step(action)
ray.shutdown()

#moderate prior precision

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'policy_loss': 1.5299568176269531, 'cur_lr': 0.0004}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 200.0, 'num_grad_updates_lifetime': 1.5, 'diff_num_grad_updates_vs_sampler_policy': 0.5}}, 'num_env_steps_sampled': 200, 'num_env_steps_trained': 200, 'num_agent_steps_sampled': 400, 'num_agent_steps_trained': 400}, 'sampler_results': {'episode_reward_max': 8.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 4.45, 'episode_len_mean': 2.0, 'episode_media': {}, 'episodes_this_iter': 100, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [7.0, 0.0, 7.0, 8.0, 7.0, 1.0, 1.0, 1.0, 1.0, 8.0, 7.0, 1.0, 7.0, 7.0, 7.0, 1.0, 1.0, 1.0, 7.0, 7.0, 0.0, 7.0, 1.0, 7.0, 1.0, 0.0, 7.0, 8.0, 1.0, 7.0, 7.0, 7.0, 8.0, 1.0, 0.0, 1.0, 1.0, 7.0, 8.0, 7.0, 7.0, 1.0, 0.0, 0.0, 1.0, 0.0, 7.

AttributeError: 'NoneType' object has no attribute 'reset'

In [61]:
algo = config.build()

# Train.
for i in range(10):
    print(algo.train())
ray.shutdown()

#prior precision is large 

2023-04-14 23:54:09,009	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-04-14 23:54:16,566	INFO trainable.py:172 -- Trainable.setup took 12.545 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'policy_loss': 1.6795865297317505, 'cur_lr': 0.0004}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 200.0, 'num_grad_updates_lifetime': 1.5, 'diff_num_grad_updates_vs_sampler_policy': 0.5}}, 'num_env_steps_sampled': 200, 'num_env_steps_trained': 200, 'num_agent_steps_sampled': 400, 'num_agent_steps_trained': 400}, 'sampler_results': {'episode_reward_max': 8.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 4.88, 'episode_len_mean': 2.0, 'episode_media': {}, 'episodes_this_iter': 100, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 1.0, 7.0, 7.0, 1.0, 1.0, 8.0, 8.0, 7.0, 7.0, 7.0, 8.0, 7.0, 7.0, 7.0, 8.0, 1.0, 7.0, 1.0, 8.0, 8.0, 0.0, 7.0, 7.0, 7.0, 1.0, 8.0, 7.0, 8.0, 1.0, 8.0, 7.0, 1.0, 0.0, 7.0, 7.0, 7.0, 1.0, 7.0, 7.0, 7.0, 1.

In [ ]:
algo = config.build()

# Train.
for i in range(10):
    print(algo.train())
ray.shutdown()

#small prior precision
#episode reward mean is 4.64 and policy loss is 1.6 under prior precision 20
#policy loss increased to 1.65 and epsidoe reward mean increased to 4.79 under prior precison 0.0000000001
#under prior precision 2 episdoe reward mean is 5.05 and loss is 1.7. second trial of same prior precison yielded 5.15 reward mean and 1.8 loss 
#trial 3 under prior precision 2 loss is 1.73 and episdoe reward mean is 5.04. trial 4 loss is 1.65 and epsidoe reward mean and reward mean 4.79
#trial 5 is 1.43 policy loss and reward of 4.18 for prior precision 2 

#reward is increasing but policy loss is also increasing

In [24]:
algo = config.build()

# Train.
for i in range(10):
    print(algo.train())
ray.shutdown()

2023-04-19 23:46:46,491	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2023-04-19 23:46:50,066	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-04-19 23:46:55,310	WARNING worker.py:846 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is because Ray does not manage GPU allocations to the driver process.
2023-04-19 23:46:55,310	WARNING env.py:53 -- Skipping env checking for this experiment
2023-04-19 23:46:55,312	WARNING algorithm_config.py:596 -- Cannot create PGConfig from given `config_dict`! Property worker_index not supported.
C:\Users\subar\anaconda3\envs\d\lib\site-packages\tensorboardX\summary.py:234: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  cum_counts = np.cumsum(np.greater(counts, 0, dty

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'policy_loss': 1.6482515335083008, 'cur_lr': 0.0004}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 200.0, 'num_grad_updates_lifetime': 1.5, 'diff_num_grad_updates_vs_sampler_policy': 0.5}}, 'num_env_steps_sampled': 200, 'num_env_steps_trained': 200, 'num_agent_steps_sampled': 400, 'num_agent_steps_trained': 400}, 'sampler_results': {'episode_reward_max': 8.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 4.79, 'episode_len_mean': 2.0, 'episode_media': {}, 'episodes_this_iter': 100, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [1.0, 7.0, 7.0, 1.0, 1.0, 7.0, 7.0, 7.0, 7.0, 0.0, 1.0, 1.0, 7.0, 7.0, 7.0, 7.0, 1.0, 8.0, 7.0, 0.0, 8.0, 1.0, 7.0, 7.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 7.0, 1.0, 7.0, 7.0, 7.0, 8.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 1.0, 8.0, 7.

In [27]:
policy = algo.get_policy()
print(policy.dist_class)

<class 'ray.rllib.models.torch.torch_action_dist.TorchCategorical'>


In [13]:
from ray.rllib.models.preprocessors import get_preprocessor

prep = get_preprocessor(algo.observation_space)#(algo.observation_space)

AttributeError: 'PG' object has no attribute 'observation_space'

In [ ]:
def predicted_models(n,X,dx):
        library_functions = [
            #lambda x : (math.e)**x,
            #lambda x : np.log(1-x),
            #lambda x : np.log(1+x),
            lambda x : np.sin(x),
            lambda x : np.cos(x),
            lambda x : 1/(1-x),
            lambda x : 1/(1-x**2),
            lambda x : 1/((1-x)**2),
            lambda x : 1/(1+x),
            lambda x : 1/(1+x**2),
            lambda x : 1/((1+x)**2),
            lambda x : 1/x,
            #lambda x : 1/x**2,
            #lambda x : 1/x**3
        ]
        library_function_names = [
            #lambda x : 'e^' + x,
    	    #lambda x : 'ln(1-'+ x + ')',
            #lambda x : 'ln(1+'+ x + ')',
    	    lambda x : 'sin(' + x + ')',
    	    lambda x : 'cos(' + x + ')',
    	    lambda x : '1/1-' + x,
    	    lambda x : '1/1-' + x + '^2',
    	    lambda x : '1/(1-' + x + ')^2',
    	    lambda x : '1/1+' + x,
    	    lambda x : '1/1+' + x + '^2',
    	    lambda x : '1/(1+' + x + ')^2',
     	    lambda x : '1/' + x,
	    #lambda x : '1/' + x + '^2',
    	    #lambda x : '1/' + x + '^3'
        ]
        
        lib_custom = ps.CustomLibrary(library_functions=library_functions, function_names=library_function_names)
        lib_poly = ps.PolynomialLibrary(degree=1, include_bias=True, include_interaction=False)
        lib = ps.GeneralizedLibrary([lib_custom, lib_poly], inputs_per_library=np.array([[0,0],[0,1]]))

        coeff = []
        for i in range(n):
            model = ps.SINDy(feature_library=lib, optimizer=Lasso(alpha=0.0001, fit_intercept=False,
            max_iter=100000), discrete_time=True)
            model.fit(X[i].T, x_dot=dx[i].T)
            coeff.append(model.coefficients())
        return np.array(coeff)

#def predicted_equations(x,parameters):
#        return parameters[0][0]/(1+x[0]**2) + parameters[0][1] + parameters[0][2]*x[0] + parameters[0][3]*x[1], 
#        parameters[1][3]*x[1] + parameters[1][2]*x[0] + parameters[1][1]

#def predicted_series(n,x,coeffs):
#        predicted_series = np.zeros_like((x))
#        for i in range(n):
#                predicted_series[i,:,0] = x[0,:,0]
#                for t in range(x.shape[2]-1):
#                        predicted_series[i,:,t+1] = predicted_equations(x[0,:,t],parameters=coeffs[i])
#        return predicted_series

def similarity(n,x,coeff,k_in):
        #Check the correlation between gorund-truth time series u-component
        corr_matrix_gt = np.corrcoef(x[:,0,:], x[:,0,:])[0:n,0:n]
        distance_matrix = pairwise_distances(coeff[:,0,:], metric='seuclidean')
        
        #similarity analysis
        s = np.sum(distance_matrix, axis=1)
        s_gt = np.sum(np.abs(corr_matrix_gt), axis=1)
        
        #predicted low-degree node
        print('one of the low degree nodes:', np.argmin(s))
        
        #predicted hub
        print('predicted hub:', np.argmax(s))
        
        #check
        print('Predicted low degree node', np.argmin(s), 'has', k_in[np.argmin(s)], 'in connection(s).')
        print('The real hub is:', np.argmax(k_in))
        
        hub_id = np.argmax(s)
        ld_id = np.argmin(s)
        
        return corr_matrix_gt, distance_matrix, s, s_gt, hub_id, ld_id

def local_dynamics_function(n,m,time,x,beta,mu,sigma):

        def rulkov_map(x):
                x = x.reshape(n,m).T
                return np.asarray([beta/(1+x[0]**2)+x[1],x[1]-mu*x[0]-sigma]).T.flatten()

        y = x.reshape(n*m,time)
        F_x = np.zeros_like(y)
        for i in range(time):
                F_x[:,i] = rulkov_map(y[:,i])
        Fx = F_x.reshape(n,m,time)
        return Fx

def coupling_effect(dx,hub_id,Fx):
        Y_hub = dx[hub_id,:,:] - Fx[hub_id,:,:-1]
        Y = dx[:,:,:] - Fx[:,:,:-1]
        return Y_hub, Y


def reconstruction(n,m,time,x,y):
        L_predicted = []
        #model = ps.SINDy(feature_library=ps.IdentityLibrary(), optimizer=ps.STLSQ(threshold=0.0001))
        model = ps.SINDy(feature_library=ps.IdentityLibrary(), optimizer=Lasso(alpha=0.001, fit_intercept=False))
        model.fit(x.reshape(n*m,time-1).T, x_dot=y.reshape(n*m,time-1).T, quiet=True)
        L_predicted.append(model.coefficients())
        L_predicted = np.array(L_predicted)[0,:,:]
        #remove kronecker product
        xxx = np.arange(0,n*m,m)
        xx, yy = np.meshgrid(xxx, xxx)
        L_predicted = L_predicted[xx,yy]
        return L_predicted

In [ ]:
#causal inference module

import numpy as np
import networkx as nx
from NEMtropy import UndirectedGraph, matrix_generator
from NEMtropy.network_functions import build_adjacency_from_edgelist
#generating null models
graph.solve_tool(model="cm_exp",
                 method="fixed_point",
                 initial_guess="random")

# After that we have solved the model we can use it to generate random versions of zachary karate club.
# The ensemble sampler function generates "n" random versions of zachary karate club using the model parameters computed by solve_tool.

graph.ensemble_sampler(10, cpu_n=12, output_dir="sample/")

In [ ]:
# memory pre-allocation
emp_dyads = np.zeros(n)
emp_singles = np.zeros(n)
emp_zeros = np.zeros(n)

for i in range(n):
    # Read the graph as an edgelist
    edgelist_ens = np.loadtxt(f"sample/{i}.txt")
    # read the graph in networkx 
    graph = nx.DiGraph()
    graph.add_edges_from(edgelist_ens.astype(int))
    # create adjacency matrix as numpy array
    a = nx.to_numpy_array(graph)
    # compute dyads numerosity
    emp_dyads[i] = count_2motif_2(a)
    emp_singles[i] = count_2motif_1(a)
    emp_zeros[i] = count_2motif_0(a)

In [ ]:
z_score = {}
emp_mu = np.mean(emp_dyads)
emp_std = np.std(emp_dyads)
z_score['dyads'] = (count_2motif_2(A) - emp_mu)/emp_std
emp_mu = np.mean(emp_singles)
emp_std = np.std(emp_singles)
z_score['singles'] = (count_2motif_1(A) - emp_mu)/emp_std
emp_mu = np.mean(emp_zeros)
emp_std = np.std(emp_zeros)
z_score['zeros'] = (count_2motif_0(A) - emp_mu)/emp_std

print(z_score)

In [ ]:
# memory pre-allocation
sample_numerosity = list(range(10,1010,50))
z_score_empirical ={
    '2': [],
    '1': [],
    '0': []
}
for n in sample_numerosity: 
    emp_dyads = np.zeros(n)
    emp_singles = np.zeros(n)
    emp_zeros = np.zeros(n)

    for i in range(n):
        # Read the graph as an edgelist
        edgelist_ens = np.loadtxt(f"sample/{i}.txt")
        # read the graph in networkx 
        graph = nx.DiGraph()
        graph.add_edges_from(edgelist_ens.astype(int))
        # create adjacency matrix as numpy array
        a = nx.to_numpy_array(graph)
        # compute dyads numerosity
        emp_dyads[i] = count_2motif_2(a)
        emp_singles[i] = count_2motif_1(a)
        emp_zeros[i] = count_2motif_0(a)

    emp_mu = np.mean(emp_dyads)
    emp_std = np.std(emp_dyads)
    z_score_empirical['2'].append((count_2motif_2(A) - emp_mu)/emp_std)
    emp_mu = np.mean(emp_singles)
    emp_std = np.std(emp_singles)
    z_score_empirical['1'].append((count_2motif_1(A) - emp_mu)/emp_std)
    emp_mu = np.mean(emp_zeros)
    emp_std = np.std(emp_zeros)
    z_score_empirical['0'].append((count_2motif_0(A) - emp_mu)/emp_std)

In [ ]:
import matplotlib.pyplot as plt

data_map = {
    '0': 'zeros',
    '1': 'singles',
    '2': 'dyads'
}

for data in ['2', '1', '0']:
    plt.figure()

    plt.plot(
        sample_numerosity,
        z_score_empirical[data],
        color = 'blue',
        marker='o', 
        linestyle='dashed',
        label = 'empirical'
        )

    plt.hlines(
        y = z_score_analytical[data],
        xmin = sample_numerosity[0],
        xmax = sample_numerosity[-1],
        color = 'red',
        label = 'analytical'
    )

    plt.xlabel('sample size')
    plt.ylabel('z score')
    plt.legend(loc='center right')
    plt.title(f'{data_map[data]}')
    plt.show()